In [ ]:
# %pip install pandas
# %pip install matplotlib
# %pip install sklearn
# %pip install scikit-learn
# %pip install tqdm

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import gc
from tqdm import tqdm
from IPython.display import display_html
import datetime as dt
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

## Selecting data

In [2]:
REGULAR_SEASON_BOXSCORES = pd.read_csv('data/gamelog-all/regular.csv')
REGULAR_SEASON_BOXSCORES
# REGULAR_SEASON_BOXSCORES.info()

,Unnamed: 0,game_season,date_game,team_id,team_id_href,date_game_href,game_location,opp_id,opp_id_href,game_result,pts,opp_pts,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,trb,ast,stl,blk,tov,pf,opp_fg,opp_fga,opp_fg_pct,opp_fg3,opp_fg3a,opp_fg3_pct,opp_ft,opp_fta,opp_ft_pct,opp_orb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,game_season_advanced,date_game_advanced,game_location_advanced,opp_id_advanced,opp_id_href_advanced,game_result_advanced,pts_advanced,opp_pts_advanced,off_rtg,def_rtg,pace,fta_per_fga_pct,fg3a_per_fga_pct,ts_pct,trb_pct,ast_pct,stl_pct,blk_pct,efg_pct,tov_pct,orb_pct,ft_rate,opp_efg_pct,opp_tov_pct,drb_pct,opp_ft_rate,net_pts,net_rtg
0,0,1,2022-10-19,ATL,/teams/ATL/2023.html,/boxscores/202210190ATL.html,NaN,HOU,/teams/HOU/2023.html,W,117,107,45,90.0,0.500,7.0,25.0,0.280,20,24.0,0.833,4.0,38.0,30.0,12.0,5.0,9.0,18.0,42,98.0,0.429,9.0,35.0,0.257,14,15.0,0.933,15.0,54.0,25.0,4.0,3.0,15.0,20.0,1,2022-10-19,NaN,HOU,/teams/HOU/2023.html,W,117,107,114.3,104.5,102.4,0.267,0.278,0.582,41.3,66.7,11.7,7.9,0.539,8.2,9.3,0.222,0.474,12.5,69.4,0.143,10,9.8
1,1,2,2022-10-21,ATL,/teams/ATL/2023.html,/boxscores/202210210ATL.html,NaN,ORL,/teams/ORL/2023.html,W,108,98,40,89.0,0.449,13.0,31.0,0.419,15,16.0,0.938,9.0,46.0,26.0,10.0,2.0,17.0,22.0,33,80.0,0.413,8.0,34.0,0.235,24,30.0,0.800,10.0,44.0,16.0,9.0,7.0,19.0,19.0,2,2022-10-21,NaN,ORL,/teams/ORL/2023.html,W,108,98,107.1,97.2,100.9,0.180,0.348,0.562,51.1,65.0,9.9,4.3,0.522,15.0,20.9,0.169,0.463,16.9,78.7,0.300,10,9.9
2,2,3,2022-10-23,ATL,/teams/ATL/2023.html,/boxscores/202210230ATL.html,NaN,CHO,/teams/CHO/2023.html,L,109,126,39,95.0,0.411,8.0,35.0,0.229,23,28.0,0.821,16.0,45.0,23.0,8.0,9.0,12.0,21.0,47,90.0,0.522,11.0,26.0,0.423,21,30.0,0.700,14.0,52.0,28.0,7.0,6.0,13.0,24.0,3,2022-10-23,NaN,CHO,/teams/CHO/2023.html,L,109,126,108.7,125.7,100.2,0.295,0.368,0.508,46.4,59.0,8.0,14.1,0.453,10.1,29.6,0.242,0.583,11.2,67.4,0.233,-17,-17.0
3,3,4,2022-10-26,ATL,/teams/ATL/2023.html,/boxscores/202210260DET.html,@,DET,/teams/DET/2023.html,W,118,113,45,91.0,0.495,8.0,22.0,0.364,20,25.0,0.800,10.0,46.0,22.0,6.0,12.0,12.0,20.0,42,91.0,0.462,14.0,32.0,0.438,15,22.0,0.682,10.0,40.0,23.0,4.0,6.0,13.0,25.0,4,2022-10-26,@,DET,/teams/DET/2023.html,W,118,113,116.8,111.8,101.0,0.275,0.242,0.578,53.5,48.9,5.9,20.3,0.538,10.5,25.0,0.220,0.538,11.4,78.3,0.165,5,5.0
4,4,5,2022-10-28,ATL,/teams/ATL/2023.html,/boxscores/202210280DET.html,@,DET,/teams/DET/2023.html,W,136,112,55,97.0,0.567,12.0,29.0,0.414,14,18.0,0.778,10.0,45.0,31.0,5.0,8.0,6.0,26.0,37,86.0,0.430,7.0,32.0,0.219,31,38.0,0.816,14.0,42.0,18.0,2.0,4.0,10.0,22.0,5,2022-10-28,@,DET,/teams/DET/2023.html,W,136,112,139.8,115.1,97.3,0.186,0.299,0.648,51.7,56.4,5.1,14.8,0.629,5.4,26.3,0.144,0.471,8.9,71.4,0.360,24,24.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135884,57,58,1950-03-01,WAT,/teams/WAT/1950.html,/boxscores/195003010WAT.html,NaN,SYR,/teams/SYR/1950.html,L,72,93,23,NaN,NaN,NaN,NaN,NaN,26,39.0,0.667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,NaN,NaN,NaN,NaN,NaN,23,33.0,0.697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58,1950-03-01,NaN,SYR,/teams/SYR/1950.html,L,72,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-21,NaN
135885,58,59,1950-03-08,WAT,/teams/WAT/1950.html,/boxscores/195003080WAT.html,NaN,DNN,/teams/DNN/1950.html,W,97,68,37,NaN,NaN,NaN,NaN,NaN,23,33.0,0.697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,NaN,NaN,NaN,NaN,NaN,16,33.0,0.485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59,1950-03-08,NaN,DNN,/teams/DNN/1950.html,W,97,68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,NaN
135886,59,60,1950-03-12,WAT,/teams/WAT/1950.html,/boxscores/195003120INO.html,@,INO,/teams/INO/1950.html,L,89,90,36,NaN,NaN,NaN,NaN,NaN,17,21.0,0.810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32,NaN,NaN,NaN,NaN,

Clean, separate and select data 

In [4]:
EAST_CONF = [
    'ATL', 'BOS', 'BRK', 'NJN', 'CHO', 'CHA', 'CHH', 'CHI', 'CLE',
    'DET', 'IND', 'MIA', 'MIL', 'NYK', 'ORL', 'PHI', 'TOR','WAS'
]
WEST_CONF = [
    'DAL', 'DEN', 'GSW', 'HOU', 'LAC', 'LAL', 'MEM', 'VAN', 'MIN', 'NOP',
    'NOK','NOH', 'OKC', 'SEA', 'PHO', 'POR', 'SAC', 'SAS', 'UTA'
]
run = 1
if run:
    DF = pd.read_csv('data/gamelog-all/regular.csv')
    DF.loc[:,'season'] = DF['team_id_href'].str.slice(-9,-5).astype(int)
    DF.loc[:,'game_result'] = DF['game_result'].str.contains('W').astype(int)
    DF.loc[:,'total_pts'] = DF['pts'] + DF['opp_pts']
    DF.rename(columns={'game_result':'win_result'},inplace=True)

    DF = DF[(DF['season'] >= 2000) & (DF['season'] <= 2022)].reset_index(drop=True) # Games in 2000-2022
    le = LabelEncoder()
    season      = DF['season']
    date_game   = pd.to_datetime(DF['date_game'])
    month       = date_game.apply(lambda x: x.month).rename('month')
    game_id     = pd.Series(le.fit_transform(DF['date_game_href']),index=DF.index,name='game_id')
    game_location = DF['game_location'].isna().astype(int)
    game_season = DF['game_season']
    team_id     = DF['team_id']
    opp_id      = DF['opp_id']
    same_conf   = (DF['team_id'].isin(WEST_CONF) == DF['opp_id'].isin(WEST_CONF)).astype(int).rename('same_conf')

    DF_GAME_META = pd.concat(
        [date_game, season, month, game_season, game_id, team_id, opp_id, game_location, same_conf],axis=1
    )    
    DF_GAME_STATS = DF.drop(
        columns = ['Unnamed: 0','game_season_advanced','date_game_advanced','game_location_advanced',
        'opp_id_advanced','opp_id_href_advanced','game_result_advanced','pts_advanced','opp_pts_advanced',
        'game_season','date_game','team_id','team_id_href',	'date_game_href','game_location','opp_id','opp_id_href','season']
    )
    DF = pd.concat([DF_GAME_META,DF_GAME_STATS],axis=1)

    DF_GAME_META.to_csv('./data/2000-2022/facts_boxscores_meta.csv')
    DF_GAME_STATS.to_csv('./data/2000-2022/facts_boxscores_stats.csv')
    DF['win_result'].to_csv('./data/2000-2022/facts_boxscores_win_result.csv')
    DF['net_pts'].to_csv('./data/2000-2022/facts_boxscores_net_pts.csv')
    DF['total_pts'].to_csv('./data/2000-2022/facts_boxscores_total_pts.csv')
    DF.to_csv('./data/2000-2022/facts_boxscores_full.csv')
    with open('./data/2000-2022/dict_game_id.pkl','wb') as f:
        pickle.dump(dict(enumerate(le.classes_)),f)

    del date_game, season, month, game_season, game_id, team_id, opp_id, game_location, same_conf
    gc.collect()
else:
    DF_GAME_META = pd.read_csv('./data/2000-2022/facts_boxscores_meta.csv').drop(columns='Unnamed: 0')
    DF_GAME_STATS = pd.read_csv('./data/2000-2022/facts_boxscores_stats.csv').drop(columns='Unnamed: 0')
    DF = pd.read_csv('./data/2000-2022/facts_boxscores_full.csv').drop(columns='Unnamed: 0')
    
    DF['date_game'] = pd.to_datetime(DF['date_game'])
    DF_GAME_META['date_game'] = pd.to_datetime(DF_GAME_META['date_game'])
    with open('./data/2000-2022/dict_game_id.pkl','rb') as f:
        dict_game_id = pickle.load(f)

print(f"[{dt.datetime.now().strftime('%H:%M:%S %d-%m-%Y ')}] cached = {not bool(run)}")

GAME_META = list(DF_GAME_META.columns)
GAME_STATS = list(DF_GAME_STATS.columns)
print(f"{GAME_META=}")
print(f"{GAME_STATS=}")

DF


[12:40:33 19-04-2023 ] cached = False
GAME_META=['date_game', 'season', 'month', 'game_season', 'game_id', 'team_id', 'opp_id', 'game_location', 'same_conf']
GAME_STATS=['win_result', 'pts', 'opp_pts', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'opp_fg', 'opp_fga', 'opp_fg_pct', 'opp_fg3', 'opp_fg3a', 'opp_fg3_pct', 'opp_ft', 'opp_fta', 'opp_ft_pct', 'opp_orb', 'opp_trb', 'opp_ast', 'opp_stl', 'opp_blk', 'opp_tov', 'opp_pf', 'off_rtg', 'def_rtg', 'pace', 'fta_per_fga_pct', 'fg3a_per_fga_pct', 'ts_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'efg_pct', 'tov_pct', 'orb_pct', 'ft_rate', 'opp_efg_pct', 'opp_tov_pct', 'drb_pct', 'opp_ft_rate', 'net_pts', 'net_rtg', 'total_pts']


,date_game,season,month,game_season,game_id,team_id,opp_id,game_location,same_conf,win_result,pts,opp_pts,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,trb,ast,stl,blk,tov,pf,opp_fg,opp_fga,opp_fg_pct,opp_fg3,opp_fg3a,opp_fg3_pct,opp_ft,opp_fta,opp_ft_pct,opp_orb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,off_rtg,def_rtg,pace,fta_per_fga_pct,fg3a_per_fga_pct,ts_pct,trb_pct,ast_pct,stl_pct,blk_pct,efg_pct,tov_pct,orb_pct,ft_rate,opp_efg_pct,opp_tov_pct,drb_pct,opp_ft_rate,net_pts,net_rtg,total_pts
0,2021-10-21,2022,10,1,26306,ATL,DAL,1,0,1,113,87,45,94.0,0.479,15.0,35.0,0.429,8,9.0,0.889,6.0,55.0,31.0,8.0,9.0,13.0,16.0,31,93.0,0.333,13.0,43.0,0.302,12,13.0,0.923,10.0,50.0,16.0,7.0,3.0,15.0,21.0,109.9,84.6,102.9,0.096,0.372,0.577,52.4,68.9,7.8,18.0,0.559,11.7,13.0,0.085,0.403,13.2,83.1,0.129,26,25.3,200
1,2021-10-23,2022,10,2,26320,ATL,CLE,0,1,0,95,101,38,99.0,0.384,10.0,34.0,0.294,9,15.0,0.600,17.0,54.0,20.0,5.0,3.0,9.0,23.0,37,89.0,0.416,7.0,28.0,0.250,20,27.0,0.741,12.0,54.0,23.0,6.0,6.0,10.0,16.0,99.3,105.6,95.7,0.152,0.343,0.450,50.0,52.6,5.2,4.9,0.434,7.9,28.8,0.091,0.455,9.0,75.5,0.225,-6,-6.3,196
2,2021-10-25,2022,10,3,26333,ATL,DET,1,1,1,122,104,46,90.0,0.511,12.0,32.0,0.375,18,21.0,0.857,10.0,49.0,24.0,11.0,3.0,13.0,19.0,40,91.0,0.440,9.0,33.0,0.273,15,18.0,0.833,11.0,36.0,26.0,7.0,6.0,14.0,15.0,123.1,105.0,99.1,0.233,0.356,0.615,57.6,52.2,11.1,5.2,0.578,11.6,28.6,0.200,0.489,12.4,78.0,0.165,18,18.1,226
3,2021-10-27,2022,10,4,26351,ATL,NOP,0,0,1,102,99,40,96.0,0.417,8.0,30.0,0.267,14,17.0,0.824,21.0,55.0,21.0,4.0,4.0,11.0,14.0,40,93.0,0.430,11.0,36.0,0.306,8,9.0,0.889,9.0,44.0,24.0,5.0,6.0,9.0,19.0,110.2,107.0,92.5,0.177,0.313,0.493,55.6,52.5,4.3,7.0,0.458,9.6,37.5,0.146,0.489,8.5,79.1,0.086,3,3.2,201
4,2021-10-28,2022,10,5,26362,ATL,WAS,0,1,0,111,122,48,88.0,0.545,6.0,21.0,0.286,9,14.0,0.643,6.0,43.0,26.0,4.0,3.0,13.0,16.0,46,99.0,0.465,13.0,34.0,0.382,17,21.0,0.810,14.0,51.0,26.0,6.0,4.0,6.0,16.0,111.9,122.9,99.2,0.159,0.239,0.589,45.7,54.2,4.0,4.6,0.580,12.1,14.0,0.102,0.530,5.3,72.5,0.172,-11,-11.0,233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,2000-04-12,2000,4,78,1126,WAS,PHI,0,1,0,84,93,32,78.0,0.410,5.0,10.0,0.500,15,17.0,0.882,17.0,50.0,20.0,11.0,8.0,22.0,27.0,35,74.0,0.473,2.0,6.0,0.333,21,35.0,0.600,12.0,38.0,22.0,19.0,3.0,13.0,18.0,94.8,105.0,88.6,0.218,0.128,0.491,56.8,62.5,12.4,11.8,0.442,20.5,39.5,0.192,0.486,12.7,73.3,0.284,-9,-10.2,177
55042,2000-04-13,2000,4,79,1130,WAS,CHI,0,1,1,109,103,39,82.0,0.476,10.0,16.0,0.625,21,26.0,0.808,17.0,38.0,21.0,7.0,1.0,11.0,26.0,40,77.0,0.519,4.0,18.0,0.222,19,23.0,0.826,11.0,34.0,27.0,6.0,5.0,11.0,22.0,130.2,123.0,83.7,0.317,0.195,0.583,52.8,53.8,8.4,1.7,0.537,10.5,42.5,0.256,0.545,11.2,65.6,0.247,6,7.2,212
55043,2000-04-15,2000,4,80,1149,WAS,MIL,1,1,0,116,120,43,93.0,0.462,12.0,23.0,0.522,18,25.0,0.720,9.0,37.0,30.0,9.0,5.0,12.0,31.0,41,80.0,0.513,9.0,14.0,0.643,29,42.0,0.690,12.0,55.0,21.0,6.0,9.0,19.0,25.0,111.0,114.8,94.7,0.269,0.247,0.558,40.2,69.8,8.6,7.6,0.527,10.3,17.3,0.194,0.569,16.2,70.0,0.363,-4,-3.8,236
55044,2000-04-18,2000,4,81,1174,WAS,BOS,1,1,0,81,114,28,76.0,0.368,3.0,10.0,0.300,22,32.0,0.688,12.0,34.0,18.0,6.0,1.0,17.0,25.0,45,84.0,0.536,8.0,22.0,0.364,16,27.0,0.593,17.0,53.0,19.0,8.0,3.0,18.0,27.0,86.4,121.6,93.8,0.421,0.132,0.450,39.1,64.3,6.4,1.6,0.388,15.9,25.0,0.289,0.583,15.8,56.4,0.190,-33,-35.2,195


Get scheduling info

In [24]:
run = 1
if run:
    def count_num_games_in_window(series,window):
        DAY = pd.Timedelta(days=1).to_numpy().astype(int)
        return np.sum(series.iloc[:-1] >= series.iloc[-1] - window*DAY)
    
    DF = pd.read_csv('./data/2000-2022/facts_boxscores_meta.csv').drop(columns='Unnamed: 0')
    team_id = DF['team_id']
    opp_id = DF['opp_id']
    date_game   = pd.to_datetime(DF['date_game'])

    game_location = DF['game_location']
    same_conf = DF['same_conf']
    b2b_game = pd.Series(index=DF.index,name='b2b_game') # 2 games in 2 days
    b3b_game = pd.Series(index=DF.index,name='b3b_game') # 3 games in 4 days
    opp_b2b_game = pd.Series(index=DF.index,name='opp_b2b_game')
    opp_b3b_game = pd.Series(index=DF.index,name='opp_b3b_game')

    for team in pd.unique(team_id):
        ordered_index = date_game[team_id == team].sort_values().index
        b2b_game.loc[ordered_index] = date_game[ordered_index].astype(int).rolling(2).apply(count_num_games_in_window,kwargs=dict(window=1)).fillna(0) # 2 games in 2 days
        b3b_game.loc[ordered_index] = (date_game[ordered_index].astype(int).rolling(4).apply(count_num_games_in_window,kwargs=dict(window=3)) >= 2).astype(int).fillna(0) # 3 games in 4 days
    for opp in pd.unique(opp_id):
        ordered_index = date_game[opp_id == opp].sort_values().index
        opp_b2b_game.loc[ordered_index] = date_game[ordered_index].astype(int).rolling(2).apply(count_num_games_in_window,kwargs=dict(window=1)).fillna(0) # 2 games in 2 days
        opp_b3b_game.loc[ordered_index] = (date_game[ordered_index].astype(int).rolling(4).apply(count_num_games_in_window,kwargs=dict(window=3)) >= 2).astype(int).fillna(0) # 3 games in 4 days
    df_scheduling = pd.concat([game_location,same_conf,b2b_game,b3b_game,opp_b2b_game,opp_b3b_game],axis=1)
    df_scheduling.to_csv('./data/2000-2022/prior_scheduling.csv')

    del DF,game_location,same_conf,b2b_game,b3b_game,opp_b2b_game,opp_b3b_game,team_id
    gc.collect()
else:
    df_scheduling = pd.read_csv('./data/2000-2022/prior_scheduling.csv').drop(columns='Unnamed: 0')
    
print(f"[{dt.datetime.now().strftime('%H:%M:%S %d-%m-%Y')}] cached = {not bool(run)}")
df_scheduling

[13:30:13 19-04-2023] cached = False


,game_location,same_conf,b2b_game,b3b_game,opp_b2b_game,opp_b3b_game
0,1,0,0.0,0.0,0.0,0.0
1,0,1,0.0,0.0,1.0,1.0
2,1,1,0.0,0.0,0.0,0.0
3,0,0,0.0,0.0,0.0,0.0
4,0,1,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...
55041,0,1,0.0,0.0,0.0,1.0
55042,0,1,1.0,1.0,0.0,0.0
55043,1,1,0.0,1.0,1.0,1.0
55044,1,1,0.0,0.0,0.0,0.0


Rolling & Expanding

In [6]:
run = 1
if run:
    # Create rolling and expanding table
    DF_GAME_META = pd.read_csv('./data/2000-2022/facts_boxscores_meta.csv').drop(columns='Unnamed: 0')
    DF_GAME_STATS = pd.read_csv('./data/2000-2022/facts_boxscores_stats.csv').drop(columns='Unnamed: 0')    
    GAME_META = list(DF_GAME_META.columns)
    GAME_STATS = list(DF_GAME_STATS.columns)
    
    df_all_seasons_last10 = pd.DataFrame() 
    df_all_seasons_cumu = pd.DataFrame()
    for season in pd.unique(DF_GAME_META['season']):
        for team_id in pd.unique(DF_GAME_META['team_id']):
            df_team = DF_GAME_STATS[(DF_GAME_META['season'] == season) & (DF_GAME_META['team_id'] == team_id)]
            df_team_last10 = df_team.rolling(10).mean().fillna(0)  # inclusive
            df_team_cumu = df_team.expanding().mean().fillna(0) # inclusive
            df_all_seasons_last10 = pd.concat([df_all_seasons_last10,df_team_last10])
            df_all_seasons_cumu = pd.concat([df_all_seasons_cumu,df_team_cumu])

    # df_all_seasons_last10 = pd.concat([DF_GAME_META,df_all_seasons_last10],axis=1)
    # df_all_seasons_cumu = pd.concat([DF_GAME_META,df_all_seasons_cumu],axis=1)

    df_all_seasons_last10.to_csv('./data/2000-2022/facts_last10_stats.csv')
    df_all_seasons_cumu.to_csv('./data/2000-2022/facts_cumu_stats.csv')

    del df_team, df_team_last10, df_team_cumu
    gc.collect()
else:
    df_all_seasons_last10 = pd.read_csv('./data/2000-2022/facts_last10_stats.csv').drop(columns='Unnamed: 0')
    df_all_seasons_cumu = pd.read_csv('./data/2000-2022/facts_cumu_stats.csv').drop(columns='Unnamed: 0')
    
print(f"[{dt.datetime.now().strftime('%H:%M:%S %d-%m-%Y ')}] cached = {not bool(run)}")
display_html(df_all_seasons_last10)
display_html(df_all_seasons_cumu)

[12:42:47 19-04-2023 ] cached = False


,win_result,pts,opp_pts,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,trb,ast,stl,blk,tov,pf,opp_fg,opp_fga,opp_fg_pct,opp_fg3,opp_fg3a,opp_fg3_pct,opp_ft,opp_fta,opp_ft_pct,opp_orb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,off_rtg,def_rtg,pace,fta_per_fga_pct,fg3a_per_fga_pct,ts_pct,trb_pct,ast_pct,stl_pct,blk_pct,efg_pct,tov_pct,orb_pct,ft_rate,opp_efg_pct,opp_tov_pct,drb_pct,opp_ft_rate,net_pts,net_rtg,total_pts
0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,0.00,0.00,0.0000,0.0,0.00,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,0.00,0.00,0.0000,0.0,0.00,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,0.00,0.00,0.0000,0.0,0.00,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,0.00,0.00,0.0000,0.0,0.00,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,0.00,0.00,0.0000,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,0.5,96.7,96.5,37.3,83.5,0.4498,4.4,12.5,0.3885,17.7,22.2,0.7976,13.0,41.4,21.6,7.0,6.0,12.4,27.6,34.3,76.6,0.4490,3.4,10.7,0.3034,24.5,31.8,0.7815,12.4,43.3,17.7,7.3,5.0,14.6,23.0,107.56,107.25,90.11,0.2687,0.1481,0.5210,48.66,58.20,7.79,8.88,0.4765,11.69,29.14,0.2144,0.4713,13.86,69.14,0.3240,0.2,0.31,193.2
55042,0.6,99.0,97.0,37.4,81.8,0.4590,5.2,12.6,0.4377,19.0,23.5,0.8169,12.8,41.0,21.9,6.8,5.6,11.9,27.4,34.8,76.4,0.4566,3.5,11.5,0.2956,23.9,30.9,0.7860,12.1,41.3,19.2,7.1,5.1,13.8,23.1,111.63,109.35,88.87,0.2873,0.1524,0.5382,49.56,58.84,7.69,8.33,0.4908,11.41,30.17,0.2319,0.4796,13.28,69.48,0.3171,2.0,2.28,196.0
55043,0.5,100.6,100.4,38.0,82.6,0.4617,5.9,13.2,0.4605,18.7,23.2,0.8139,12.4,39.2,22.1,7.5,5.5,11.8,27.2,36.2,76.4,0.4741,3.7,11.2,0.3187,24.3,31.1,0.7925,12.0,42.4,19.3,7.0,5.3,14.3,22.8,112.24,111.81,88.81,0.2813,0.1571,0.5426,48.02,58.25,8.34,8.14,0.4970,11.26,28.95,0.2266,0.4984,13.65,68.84,0.3221,0.2,0.43,201.0
55044,0.4,99.4,103.2,37.2,82.3,0.4529,5.9,12.8,0.4691,19.1,24.4,0.7927,12.1,37.9,22.7,7.4,5.1,12.3,27.3,37.4,76.9,0.4859,3.8,11.4,0.3201,24.6,32.2,0.7705,12.4,43.6,19.4,7.3,5.2,15.0,23.1,109.62,113.55,89.93,0.2981,0.1526,0.5346,46.59,61.35,8.13,7.45,0.4883,11.65,27.96,0.2327,0.5105,14.10,67.37,0.3246,-3.8,-3.93,202.6


,win_result,pts,opp_pts,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,trb,ast,stl,blk,tov,pf,opp_fg,opp_fga,opp_fg_pct,opp_fg3,opp_fg3a,opp_fg3_pct,opp_ft,opp_fta,opp_ft_pct,opp_orb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,off_rtg,def_rtg,pace,fta_per_fga_pct,fg3a_per_fga_pct,ts_pct,trb_pct,ast_pct,stl_pct,blk_pct,efg_pct,tov_pct,orb_pct,ft_rate,opp_efg_pct,opp_tov_pct,drb_pct,opp_ft_rate,net_pts,net_rtg,total_pts
0,1.000000,113.000000,87.000000,45.000000,94.000000,0.479000,15.000000,35.000000,0.429000,8.000000,9.000000,0.889000,6.000000,55.000000,31.000000,8.000000,9.000000,13.000000,16.000000,31.000000,93.000000,0.333000,13.000000,43.000000,0.302000,12.000000,13.000000,0.923000,10.000000,50.000000,16.000000,7.000000,3.000000,15.000000,21.000000,109.900000,84.600000,102.900000,0.096000,0.372000,0.577000,52.400000,68.900000,7.800000,18.000000,0.559000,11.700000,13.000000,0.085000,0.403000,13.200000,83.100000,0.129000,26.000000,25.300000,200.000000
1,0.500000,104.000000,94.000000,41.500000,96.500000,0.431500,12.500000,34.500000,0.361500,8.500000,12.000000,0.744500,11.500000,54.500000,25.500000,6.500000,6.000000,11.000000,19.500000,34.000000,91.000000,0.374500,10.000000,35.500000,0.276000,16.000000,20.000000,0.832000,11.000000,52.000000,19.500000,6.500000,4.500000,12.500000,18.500000,104.600000,95.100000,99.300000,0.124000,0.357500,0.513500,51.200000,60.750000,6.500000,11.450000,0.496500,9.800000,20.900000,0.088000,0.429000,11.100000,79.300000,0.177000,10.000000,9.500000,198.000000
2,0.666667,110.000000,97.333333,43.000000,94.333333,0.458000,12.333333,33.666667,0.366000,11.666667,15.000000,0.782000,11.000000,52.666667,25.000000,8.000000,5.000000,11.666667,19.333333,36.000000,91.000000,0.396333,9.666667,34.666667,0.275000,15.666667,19.333333,0.832333,11.000000,46.666667,21.666667,6.666667,5.000000,13.000000,17.333333,110.766667,98.400000,99.233333,0.160333,0.357000,0.547333,53.333333,57.900000,8.033333,9.366667,0.523667,10.400000,23.466667,0.125333,0.449000,11.533333,78.866667,0.173000,12.666667,12.366667,207.333333
3,0.750000,108.000000,97.750000,42.250000,94.750000,0.447750,11.250000,32.750000,0.341250,12.250000,15.500000,0.792500,13.500000,53.250000,24.000000,7.000000,4.750000,11.500000,18.000000,37.000000,91.500000,0.404750,10.000000,35.000000,0.282750,13.750000,16.750000,0.846500,10.500000,46.000000,22.250000,6.250000,5.250000,12.000000,17.750000,110.625000,100.550000,97.550000,0.164500,0.346000,0.533750,53.900000,56.550000,7.100000,8.775000,0.507250,10.200000,26.975000,0.130500,0.459000,10.775000,78.925000,0.151250,10.250000,10.075000,205.750000
4,0.600000,108.600000,102.600000,43.400000,93.400000,0.467200,10.200000,30.400000,0.330200,11.600000,15.200000,0.762600,12.000000,51.200000,24.400000,6.400000,4.400000,11.800000,17.600000,38.800000,93.000000,0.416800,10.600000,34.800000,0.302600,14.400000,17.600000,0.839200,11.200000,47.000000,23.000000,6.200000,5.000000,10.800000,17.400000,110.880000,105.020000,97.880000,0.163400,0.324600,0.544800,52.260000,56.080000,6.480000,7.940000,0.521800,10.580000,24.380000,0.124800,0.473200,9.680000,77.640000,0.155400,6.000000,5.860000,211.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,0.358974,96.282051,99.269231,36.679487,81.294872,0.453192,3.858974,10.576923,0.363808,19.064103,25.666667,0.745077,12.961538,42.935897,21.500000,7.269231,4.782051,15.589744,26.115385,36.371795,79.820513,0.456038,4.705128,12.730769,0.365590,21.820513,29.243590,0.747282,11.705128,41.000000,21.820513,8.435897,6.141026,14.474359,23.987179,103.958974,107.060256,92.196154,0.320962,0.129872,0.521474,51.146154,58.724359,7.857692,7.117949,0.476936,14.374359,30.311538,0.238423,0.485705,13.507692,72.112821,0.278141,-2.987179,-3.101282,195.551282
55042,0.367089,96.443038,99.316456,36.708861,81.303797,0.453481,3.936709,10.6

Build Index of Prev entry

- April 18: Added h2h_index

In [7]:
run = 1
if run:
    DF = pd.read_csv('./data/2000-2022/facts_boxscores_meta.csv').drop(columns='Unnamed: 0')
    df_all_seasons_index = DF
    df_all_seasons_index.insert(len(df_all_seasons_index.columns),'team_prev_game_id',-1)
    df_all_seasons_index.insert(len(df_all_seasons_index.columns),'opp_prev_game_id',-1)
    df_all_seasons_index.insert(len(df_all_seasons_index.columns),'h2h_prev_game_id',-1)
    for season in tqdm(pd.unique(df_all_seasons_index['season'])):
        df_season = df_all_seasons_index[(df_all_seasons_index['season'] == season)]
        for team_id in pd.unique(df_season['team_id']):
            df_team = df_season[df_season['team_id'] == team_id]
            df_team.loc[:,'team_prev_game_id'] = [-1] + list(df_team['game_id'][:-1])
            df_season.loc[df_team.index,:] = df_team

            for opp_id in pd.unique(df_season['opp_id']):
                df_h2h = df_season[(df_season['team_id'] == team_id) & (df_season['opp_id'] == opp_id)].sort_values('date_game')
                df_h2h.loc[:,'h2h_prev_game_id'] = [-1] + list(df_h2h['game_id'])[:-1]
                df_season.loc[df_h2h.index,:] = df_h2h
        
        for game_id in pd.unique(df_season['game_id']):
            df_game_id = df_season[df_season['game_id'] == game_id]
            df_game_id.loc[:,'opp_prev_game_id'] = df_game_id['team_prev_game_id'].values[::-1]
            df_season.loc[df_game_id.index,:] = df_game_id
           
        df_all_seasons_index.loc[df_season.index] = df_season
    df_all_seasons_index['game_index'] = pd.Index(map(tuple,df_all_seasons_index[['game_id','team_id']].values))
    df_all_seasons_index['team_prev_game_index'] = pd.Index(map(tuple,df_all_seasons_index[['team_prev_game_id','team_id']].values))
    df_all_seasons_index['opp_prev_game_index'] = pd.Index(map(tuple,df_all_seasons_index[['opp_prev_game_id','opp_id']].values))
    df_all_seasons_index['h2h_prev_game_index'] = pd.Index(map(tuple,df_all_seasons_index[['h2h_prev_game_id','team_id']].values))
    df_all_seasons_index['opp_h2h_prev_game_index'] = pd.Index(map(tuple,df_all_seasons_index[['h2h_prev_game_id','opp_id']].values))
    df_all_seasons_index.to_csv('./data/2000-2022/facts_game_index.csv')

    del df_team, df_season, df_game_id
    gc.collect()

else:
    df_all_seasons_index = pd.read_csv('./data/2000-2022/facts_game_index.csv').drop(columns='Unnamed: 0')
    df_all_seasons_index['game_index'] = pd.Index(map(tuple,df_all_seasons_index[['game_id','team_id']].values))
    df_all_seasons_index['team_prev_game_index'] = pd.Index(map(tuple,df_all_seasons_index[['team_prev_game_id','team_id']].values))
    df_all_seasons_index['opp_prev_game_index'] = pd.Index(map(tuple,df_all_seasons_index[['opp_prev_game_id','opp_id']].values))
    df_all_seasons_index['h2h_prev_game_index'] = pd.Index(map(tuple,df_all_seasons_index[['h2h_prev_game_id','team_id']].values))
    df_all_seasons_index['opp_h2h_prev_game_index'] = pd.Index(map(tuple,df_all_seasons_index[['h2h_prev_game_id','opp_id']].values))

print(f"[{dt.datetime.now().strftime('%H:%M:%S %d-%m-%Y')}] cached = {not bool(run)}")
display_html(df_all_seasons_index)


100%|██████████| 23/23 [00:46<00:00,  2.02s/it]


[12:44:30 19-04-2023] cached = False


,date_game,season,month,game_season,game_id,team_id,opp_id,game_location,same_conf,team_prev_game_id,opp_prev_game_id,h2h_prev_game_id,game_index,team_prev_game_index,opp_prev_game_index,h2h_prev_game_index,opp_h2h_prev_game_index
0,2021-10-21,2022,10,1,26306,ATL,DAL,1,0,-1,-1,-1,"(26306, ATL)","(-1, ATL)","(-1, DAL)","(-1, ATL)","(-1, DAL)"
1,2021-10-23,2022,10,2,26320,ATL,CLE,0,1,26306,26311,-1,"(26320, ATL)","(26306, ATL)","(26311, CLE)","(-1, ATL)","(-1, CLE)"
2,2021-10-25,2022,10,3,26333,ATL,DET,1,1,26320,26319,-1,"(26333, ATL)","(26320, ATL)","(26319, DET)","(-1, ATL)","(-1, DET)"
3,2021-10-27,2022,10,4,26351,ATL,NOP,0,0,26333,26340,-1,"(26351, ATL)","(26333, ATL)","(26340, NOP)","(-1, ATL)","(-1, NOP)"
4,2021-10-28,2022,10,5,26362,ATL,WAS,0,1,26351,26347,-1,"(26362, ATL)","(26351, ATL)","(26347, WAS)","(-1, ATL)","(-1, WAS)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,2000-04-12,2000,4,78,1126,WAS,PHI,0,1,1113,1110,827,"(1126, WAS)","(1113, WAS)","(1110, PHI)","(827, WAS)","(827, PHI)"
55042,2000-04-13,2000,4,79,1130,WAS,CHI,0,1,1126,1114,945,"(1130, WAS)","(1126, WAS)","(1114, CHI)","(945, WAS)","(945, CHI)"
55043,2000-04-15,2000,4,80,1149,WAS,MIL,1,1,1130,1140,761,"(1149, WAS)","(1130, WAS)","(1140, MIL)","(761, WAS)","(761, MIL)"
55044,2000-04-18,2000,4,81,1174,WAS,BOS,1,1,1149,1151,1032,"(1174, WAS)","(1149, WAS)","(1151, BOS)","(1032, WAS)","(1032, BOS)"


Make Prior Tables
- Stub (Game meta)
- Team Prior
- Opp Prior

In [10]:
df_all_seasons_cumu

,win_result,pts,opp_pts,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,trb,ast,stl,blk,tov,pf,opp_fg,opp_fga,opp_fg_pct,opp_fg3,opp_fg3a,opp_fg3_pct,opp_ft,opp_fta,opp_ft_pct,opp_orb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,off_rtg,def_rtg,pace,fta_per_fga_pct,fg3a_per_fga_pct,ts_pct,trb_pct,ast_pct,stl_pct,blk_pct,efg_pct,tov_pct,orb_pct,ft_rate,opp_efg_pct,opp_tov_pct,drb_pct,opp_ft_rate,net_pts,net_rtg,total_pts
0,1.000000,113.000000,87.000000,45.000000,94.000000,0.479000,15.000000,35.000000,0.429000,8.000000,9.000000,0.889000,6.000000,55.000000,31.000000,8.000000,9.000000,13.000000,16.000000,31.000000,93.000000,0.333000,13.000000,43.000000,0.302000,12.000000,13.000000,0.923000,10.000000,50.000000,16.000000,7.000000,3.000000,15.000000,21.000000,109.900000,84.600000,102.900000,0.096000,0.372000,0.577000,52.400000,68.900000,7.800000,18.000000,0.559000,11.700000,13.000000,0.085000,0.403000,13.200000,83.100000,0.129000,26.000000,25.300000,200.000000
1,0.500000,104.000000,94.000000,41.500000,96.500000,0.431500,12.500000,34.500000,0.361500,8.500000,12.000000,0.744500,11.500000,54.500000,25.500000,6.500000,6.000000,11.000000,19.500000,34.000000,91.000000,0.374500,10.000000,35.500000,0.276000,16.000000,20.000000,0.832000,11.000000,52.000000,19.500000,6.500000,4.500000,12.500000,18.500000,104.600000,95.100000,99.300000,0.124000,0.357500,0.513500,51.200000,60.750000,6.500000,11.450000,0.496500,9.800000,20.900000,0.088000,0.429000,11.100000,79.300000,0.177000,10.000000,9.500000,198.000000
2,0.666667,110.000000,97.333333,43.000000,94.333333,0.458000,12.333333,33.666667,0.366000,11.666667,15.000000,0.782000,11.000000,52.666667,25.000000,8.000000,5.000000,11.666667,19.333333,36.000000,91.000000,0.396333,9.666667,34.666667,0.275000,15.666667,19.333333,0.832333,11.000000,46.666667,21.666667,6.666667,5.000000,13.000000,17.333333,110.766667,98.400000,99.233333,0.160333,0.357000,0.547333,53.333333,57.900000,8.033333,9.366667,0.523667,10.400000,23.466667,0.125333,0.449000,11.533333,78.866667,0.173000,12.666667,12.366667,207.333333
3,0.750000,108.000000,97.750000,42.250000,94.750000,0.447750,11.250000,32.750000,0.341250,12.250000,15.500000,0.792500,13.500000,53.250000,24.000000,7.000000,4.750000,11.500000,18.000000,37.000000,91.500000,0.404750,10.000000,35.000000,0.282750,13.750000,16.750000,0.846500,10.500000,46.000000,22.250000,6.250000,5.250000,12.000000,17.750000,110.625000,100.550000,97.550000,0.164500,0.346000,0.533750,53.900000,56.550000,7.100000,8.775000,0.507250,10.200000,26.975000,0.130500,0.459000,10.775000,78.925000,0.151250,10.250000,10.075000,205.750000
4,0.600000,108.600000,102.600000,43.400000,93.400000,0.467200,10.200000,30.400000,0.330200,11.600000,15.200000,0.762600,12.000000,51.200000,24.400000,6.400000,4.400000,11.800000,17.600000,38.800000,93.000000,0.416800,10.600000,34.800000,0.302600,14.400000,17.600000,0.839200,11.200000,47.000000,23.000000,6.200000,5.000000,10.800000,17.400000,110.880000,105.020000,97.880000,0.163400,0.324600,0.544800,52.260000,56.080000,6.480000,7.940000,0.521800,10.580000,24.380000,0.124800,0.473200,9.680000,77.640000,0.155400,6.000000,5.860000,211.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,0.358974,96.282051,99.269231,36.679487,81.294872,0.453192,3.858974,10.576923,0.363808,19.064103,25.666667,0.745077,12.961538,42.935897,21.500000,7.269231,4.782051,15.589744,26.115385,36.371795,79.820513,0.456038,4.705128,12.730769,0.365590,21.820513,29.243590,0.747282,11.705128,41.000000,21.820513,8.435897,6.141026,14.474359,23.987179,103.958974,107.060256,92.196154,0.320962,0.129872,0.521474,51.146154,58.724359,7.857692,7.117949,0.476936,14.374359,30.311538,0.238423,0.485705,13.507692,72.112821,0.278141,-2.987179,-3.101282,195.551282
55042,0.367089,96.443038,99.316456,36.708861,81.303797,0.453481,3.936709,10.6

In [17]:
run = 1
if run:
    DF_GAME_META = pd.read_csv('./data/2000-2022/facts_boxscores_meta.csv').drop(columns='Unnamed: 0')
    df_all_seasons_last10 = pd.read_csv('./data/2000-2022/facts_last10_stats.csv').drop(columns='Unnamed: 0')
    df_all_seasons_cumu = pd.read_csv('./data/2000-2022/facts_cumu_stats.csv').drop(columns='Unnamed: 0')
    df_all_seasons_index = pd.read_csv('./data/2000-2022/facts_game_index.csv').drop(columns='Unnamed: 0')
    game_index = pd.Index(map(tuple,df_all_seasons_index[['game_id','team_id']].values))
    team_prev_game_index = pd.Index(map(tuple,df_all_seasons_index[['team_prev_game_id','team_id']].values))
    opp_prev_game_index = pd.Index(map(tuple,df_all_seasons_index[['opp_prev_game_id','opp_id']].values))
    # df_all_seasons_index['h2h_prev_game_index'] = pd.Index(map(tuple,df_all_seasons_index[['h2h_prev_game_id','team_id']].values))
    # df_all_seasons_index['opp_h2h_prev_game_index'] = pd.Index(map(tuple,df_all_seasons_index[['h2h_prev_game_id','opp_id']].values))
    # Should implement h2h stats later on
    # Strength of Schedule?

    DF_GAME_META = pd.read_csv('./data/2000-2022/facts_boxscores_meta.csv').drop(columns='Unnamed: 0')
    DF_GAME_STATS = pd.read_csv('./data/2000-2022/facts_boxscores_stats.csv').drop(columns='Unnamed: 0')

    for data_type,df in zip(['cumu','last10'],[df_all_seasons_cumu, df_all_seasons_last10]):
        # df = df.set_index(['game_id','team_id'],drop=False)
        df = df.set_index(game_index)
        for team_id in pd.unique(DF_GAME_META['team_id']):
            df.loc[(-1,team_id),:] = 0    # Create dummy index/entries

        # stub = DF_GAME_META.copy().reset_index(drop=True)
        team_prior = df.loc[team_prev_game_index,GAME_STATS].reset_index(drop=True)
        opp_prior = df.loc[opp_prev_game_index,GAME_STATS].reset_index(drop=True)
        net_prior = team_prior - opp_prior
        avg_prior = (team_prior + opp_prior)/2
        team_prior.columns = team_prior.columns + '_team'
        opp_prior.columns = opp_prior.columns +  '_opp'
        net_prior.columns = net_prior.columns + '_net'
        avg_prior.columns = avg_prior.columns +  '_avg'
        team_prior.to_csv(f'./data/2000-2022/prior_{data_type}_stats_team.csv')
        opp_prior.to_csv(f'./data/2000-2022/prior_{data_type}_stats_opp.csv')
        net_prior.to_csv(f'./data/2000-2022/prior_{data_type}_stats_net.csv')
        avg_prior.to_csv(f'./data/2000-2022/prior_{data_type}_stats_avg.csv')

else:
    data_type = 'cumu'
    team_prior = pd.read_csv(f'./data/2000-2022/prior_{data_type}_stats_team.csv').drop(columns='Unnamed: 0')
    opp_prior = pd.read_csv(f'./data/2000-2022/prior_{data_type}_stats_opp.csv').drop(columns='Unnamed: 0')
    net_prior = pd.read_csv(f'./data/2000-2022/prior_{data_type}_stats_net.csv').drop(columns='Unnamed: 0')
    avg_prior = pd.read_csv(f'./data/2000-2022/prior_{data_type}_stats_avg.csv').drop(columns='Unnamed: 0')
print(f"[{dt.datetime.now().strftime('%H:%M:%S %d-%m-%Y')}] cached = {not bool(run)}")
display_html(team_prior)
display_html(net_prior)

[13:05:58 19-04-2023] cached = False


,win_result_team,pts_team,opp_pts_team,fg_team,fga_team,fg_pct_team,fg3_team,fg3a_team,fg3_pct_team,ft_team,fta_team,ft_pct_team,orb_team,trb_team,ast_team,stl_team,blk_team,tov_team,pf_team,opp_fg_team,opp_fga_team,opp_fg_pct_team,opp_fg3_team,opp_fg3a_team,opp_fg3_pct_team,opp_ft_team,opp_fta_team,opp_ft_pct_team,opp_orb_team,opp_trb_team,opp_ast_team,opp_stl_team,opp_blk_team,opp_tov_team,opp_pf_team,off_rtg_team,def_rtg_team,pace_team,fta_per_fga_pct_team,fg3a_per_fga_pct_team,ts_pct_team,trb_pct_team,ast_pct_team,stl_pct_team,blk_pct_team,efg_pct_team,tov_pct_team,orb_pct_team,ft_rate_team,opp_efg_pct_team,opp_tov_pct_team,drb_pct_team,opp_ft_rate_team,net_pts_team,net_rtg_team,total_pts_team
0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,0.00,0.00,0.0000,0.0,0.00,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,0.00,0.00,0.0000,0.0,0.00,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,0.00,0.00,0.0000,0.0,0.00,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,0.00,0.00,0.0000,0.0,0.00,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,0.00,0.00,0.0000,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,0.6,98.8,96.5,38.2,84.6,0.4549,4.5,12.9,0.3814,17.9,22.7,0.7867,12.7,41.0,22.0,6.8,5.8,11.4,27.1,34.0,77.1,0.4422,3.9,12.1,0.3051,24.6,31.3,0.7948,12.6,44.1,17.1,6.2,5.5,14.8,23.4,109.44,106.81,90.49,0.2716,0.1510,0.5251,47.98,57.80,7.52,8.72,0.4817,10.72,28.23,0.2143,0.4676,13.99,68.77,0.3234,2.3,2.63,195.3
55042,0.5,96.7,96.5,37.3,83.5,0.4498,4.4,12.5,0.3885,17.7,22.2,0.7976,13.0,41.4,21.6,7.0,6.0,12.4,27.6,34.3,76.6,0.4490,3.4,10.7,0.3034,24.5,31.8,0.7815,12.4,43.3,17.7,7.3,5.0,14.6,23.0,107.56,107.25,90.11,0.2687,0.1481,0.5210,48.66,58.20,7.79,8.88,0.4765,11.69,29.14,0.2144,0.4713,13.86,69.14,0.3240,0.2,0.31,193.2
55043,0.6,99.0,97.0,37.4,81.8,0.4590,5.2,12.6,0.4377,19.0,23.5,0.8169,12.8,41.0,21.9,6.8,5.6,11.9,27.4,34.8,76.4,0.4566,3.5,11.5,0.2956,23.9,30.9,0.7860,12.1,41.3,19.2,7.1,5.1,13.8,23.1,111.63,109.35,88.87,0.2873,0.1524,0.5382,49.56,58.84,7.69,8.33,0.4908,11.41,30.17,0.2319,0.4796,13.28,69.48,0.3171,2.0,2.28,196.0
55044,0.5,100.6,100.4,38.0,82.6,0.4617,5.9,13.2,0.4605,18.7,23.2,0.8139,12.4,39.2,22.1,7.5,5.5,11.8,27.2,36.2,76.4,0.4741,3.7,11.2,0.3187,24.3,31.1,0.7925,12.0,42.4,19.3,7.0,5.3,14.3,22.8,112.24,111.81,88.81,0.2813,0.1571,0.5426,48.02,58.25,8.34,8.14,0.4970,11.26,28.95,0.2266,0.4984,13.65,68.84,0.3221,0.2,0.43,201.0


,win_result_net,pts_net,opp_pts_net,fg_net,fga_net,fg_pct_net,fg3_net,fg3a_net,fg3_pct_net,ft_net,fta_net,ft_pct_net,orb_net,trb_net,ast_net,stl_net,blk_net,tov_net,pf_net,opp_fg_net,opp_fga_net,opp_fg_pct_net,opp_fg3_net,opp_fg3a_net,opp_fg3_pct_net,opp_ft_net,opp_fta_net,opp_ft_pct_net,opp_orb_net,opp_trb_net,opp_ast_net,opp_stl_net,opp_blk_net,opp_tov_net,opp_pf_net,off_rtg_net,def_rtg_net,pace_net,fta_per_fga_pct_net,fg3a_per_fga_pct_net,ts_pct_net,trb_pct_net,ast_pct_net,stl_pct_net,blk_pct_net,efg_pct_net,tov_pct_net,orb_pct_net,ft_rate_net,opp_efg_pct_net,opp_tov_pct_net,drb_pct_net,opp_ft_rate_net,net_pts_net,net_rtg_net,total_pts_net
0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,0.00,0.00,0.0000,0.0,0.00,0.0
1,-0.5,-109.8,-101.0,-40.2,-86.0,-0.4694,-12.1,-32.6,-0.3690,-17.3,-23.3,-0.7363,-9.2,-45.6,-28.6,-10.1,-4.5,-15.7,-20.6,-37.8,-85.8,-0.4404,-9.2,-27.5,-0.3420,-16.2,-21.2,-0.7834,-8.9,-42.2,-19.3,-9.0,-5.8,-16.2,-21.7,-109.43,-100.51,-100.39,-0.2722,-0.3796,-0.5731,-52.08,-70.83,-10.05,-7.89,-0.5404,-14.01,-21.58,-0.2017,-0.4950,-14.54,-80.54,-0.1905,-8.8,-8.92,-210.8
2,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000,0.00,0.00,0.0000,0.0,0.00,0.0
3,-0.3,-93.4,-94.1,-34.5,-78.9,-0.4378,-6.2,-17.5,-0.3525,-18.2,-22.0,-0.8353,-8.2,-39.2,-20.7,-6.4,-5.2,-10.8,-19.4,-34.8,-82.7,-0.4215,-8.8,-23.6,-0.3629,-15.7,-20.0,-0.7922,-13.8,-47.4,-21.5,-4.8,-4.3,-13.7,-20.0,-105.16,-106.09,-88.91,-0.2853,-0.2256,-0.5291,-45.18,-60.69,-7.19,-8.67,-0.4779,-10.81,-19.26,-0.2361,-0.4751,-12.96,-69.73,-0.1905,0.7,0.93,-187.5
4,-0.5,-100.8,-99.8,-38.7,-84.6,-0.4576,-5.8,-14.7,-0.3987,-17.6,-22.1,-0.7903,-11.6,-43.0,-23.3,-6.9,-5.5,-12.1,-24.0,-36.8,-84.6,-0.4372,-5.3,-16.4,-0.3048,-20.9,-28.4,-0.7183,-14.5,-47.4,-22.3,-7.2,-5.2,-13.7,-21.9,-108.55,-107.30,-93.09,-0.2650,-0.1737,-0.5343,-47.52,-60.30,-7.42,-8.06,-0.4920,-11.39,-26.10,-0.2105,-0.4684,-12.32,-68.33,-0.2519,-1.0,-1.25,-200.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,0.3,1.7,-6.0,2.7,7.4,-0.0042,0.5,1.1,0.0465,-4.2,-6.4,0.0232,1.7,1.3,0.3,0.5,2.9,-2.2,7.6,-5.8,-6.0,-0.0395,-2.0,-3.6,-0.0609,7.6,9.7,0.0208,1.1,3.2,-9.4,-0.7,2.0,1.9,-0.9,2.72,-6.19,-0.39,-0.1110,0.0000,-0.0139,-1.31,-3.07,0.69,4.45,-0.0026,-2.41,1.42,-0.0775,-0.0498,1.88,-3.43,0.1137,7.7,8.91,-4.3
55042,0.0,-12.1,-8.7,-3.6,-2.1,-0.0280,-6.5,-19.3,0.0542,1.6,0.4,0.0587,3.8,-2.6,-3.9,-2.9,1.1,-2.9,7.6,-6.7,-11.0,-0.0191,-5.5,-14.3,-0.0605,10.2,12.8,0.0156,3.4,0.6,-2.7,-1.3,-0.3,-1.2,2.5,-1.01,1.91,-9.98,0.0154,-0.2242,-0.0499,-2.20,-3.93,-2.06,0.97,-0.0652,-2.16,7.79,0.0274,-0.0476,-0.19,-10.94,0.1597,-3.4,-2.92,-20.8
55043,0.2,-2.8,-7.9,0.7,-1.7,0.0204,-0.5,-1.7,0.0280,-3.7,-5.2,0.0239,-0.2,-4.4,3.6,-0.5,0.1,-2.0,8.3,-4.7,-10.2,-0.0007,-3.6,-9.9,-0.0271,5.1,8.5,-0.0565,-0.5,-3.3,-0.8,-0.9,-0.6,1.0,0.3,4.10,-1.99,-3.68,-0.0606,-0.0199,0.0101,-0.93,8.69,0.00,-0.01,0.0182,-1.13,1.60,-0.0429,-0.0188,1.59,-2.73,0.0996,5.1,6.09,-10.7
55044,0.5,100.6,100.4,38.0,82.6,0.4617,5.9,13.2,0.4605,18.7,23.2,0.8139,12.4,39.2,22.1,7.5,5.5,11.8,27.2,36.2,76.4,0.4741,3.7,11.2,0.3187,24.3,31.1,0.7925,12.0,42.4,19.3,7.0,5.3,14.3,22.8,112.24,111.81,88.81,0.2813,0.1571,0.5426,48.02,58.25,8.34,8.14,0.4970,11.26,28.95,0.2266,0.4984,13.65,68.84,0.3221,0.2,0.43,201.0


Now check to see if prior stats correlates well with actual stats

April 17: Add Per round ranking through each game

In [18]:
run = 1
if run:
    DF_GAME_META = pd.read_csv('./data/2000-2022/facts_boxscores_meta.csv').drop(columns='Unnamed: 0')
    DF_GAME_STATS = pd.read_csv('./data/2000-2022/facts_boxscores_stats.csv').drop(columns='Unnamed: 0')
    GAME_STATS = DF_GAME_STATS.columns
    GAME_STATS_TEAM = GAME_STATS[~GAME_STATS.str.startswith('opp_')]
    GAME_STATS_OPP = GAME_STATS[GAME_STATS.str.startswith('opp_')]
    df_all_seasons_cumu = pd.read_csv('./data/2000-2022/facts_cumu_stats.csv').drop(columns='Unnamed: 0')

    df_lg_rank = df_all_seasons_cumu[GAME_STATS].copy()
    for season in tqdm(pd.unique(DF_GAME_META['season'])):
        for game_season in pd.unique(DF_GAME_META[(DF_GAME_META['season'] == season)]['game_season']):
            filters = (DF_GAME_META['season'] == season) & (DF_GAME_META['game_season'] == game_season)
            df_lg_rank.loc[filters,GAME_STATS_TEAM] = df_lg_rank.loc[filters,GAME_STATS_TEAM].rank(axis=0,method='min',ascending=False) # Rank stats descendingly (1 -> Highest value)
            df_lg_rank.loc[filters,GAME_STATS_OPP] = df_lg_rank.loc[filters,GAME_STATS_OPP].rank(axis=0,method='min',ascending=True) # Rank stats ascendingly (1 -> Lowest value)
    df_lg_rank.columns = df_lg_rank.columns + '_rank'
    # df_lg_rank = pd.concat([DF_GAME_META,df_lg_rank],axis=1)
    df_lg_rank.to_csv('./data/2000-2022/facts_cumu_stats_rankings.csv')
else:
    df_lg_rank = pd.read_csv('./data/2000-2022/facts_cumu_stats_rankings.csv').drop(columns='Unnamed: 0')
    
print(f"[{dt.datetime.now().strftime('%H:%M:%S %d-%m-%Y')}] cached = {not bool(run)}")
df_lg_rank


100%|██████████| 23/23 [00:02<00:00, 10.57it/s]


[13:14:12 19-04-2023] cached = False


,win_result_rank,pts_rank,opp_pts_rank,fg_rank,fga_rank,fg_pct_rank,fg3_rank,fg3a_rank,fg3_pct_rank,ft_rank,fta_rank,ft_pct_rank,orb_rank,trb_rank,ast_rank,stl_rank,blk_rank,tov_rank,pf_rank,opp_fg_rank,opp_fga_rank,opp_fg_pct_rank,opp_fg3_rank,opp_fg3a_rank,opp_fg3_pct_rank,opp_ft_rank,opp_fta_rank,opp_ft_pct_rank,opp_orb_rank,opp_trb_rank,opp_ast_rank,opp_stl_rank,opp_blk_rank,opp_tov_rank,opp_pf_rank,off_rtg_rank,def_rtg_rank,pace_rank,fta_per_fga_pct_rank,fg3a_per_fga_pct_rank,ts_pct_rank,trb_pct_rank,ast_pct_rank,stl_pct_rank,blk_pct_rank,efg_pct_rank,tov_pct_rank,orb_pct_rank,ft_rate_rank,opp_efg_pct_rank,opp_tov_pct_rank,drb_pct_rank,opp_ft_rate_rank,net_pts_rank,net_rtg_rank,total_pts_rank
0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,30.0,30.0,1.0,30.0,2.0,1.0,8.0,1.0,25.0,30.0,2.0,29.0,1.0,30.0,30.0,4.0,1.0,1.0,30.0,7.0,28.0,1.0,6.0,1.0,24.0,16.0,5.0,30.0,1.0,30.0,4.0,1.0,1.0,1.0,15.0,1.0,1.0,27.0,30.0,30.0,1.0,19.0,1.0,1.0,1.0,1.0,14.0
1,11.0,10.0,4.0,2.0,1.0,28.0,1.0,2.0,15.0,30.0,30.0,19.0,9.0,1.0,2.0,21.0,8.0,30.0,27.0,2.0,29.0,1.0,24.0,30.0,2.0,3.0,2.0,30.0,12.0,30.0,6.0,2.0,8.0,6.0,1.0,22.0,30.0,4.0,30.0,4.0,28.0,5.0,9.0,27.0,2.0,15.0,30.0,26.0,30.0,1.0,4.0,4.0,2.0,1.0,2.0,16.0
2,4.0,2.0,8.0,2.0,1.0,11.0,1.0,2.0,12.0,30.0,30.0,7.0,15.0,1.0,2.0,7.0,22.0,29.0,27.0,5.0,29.0,1.0,23.0,30.0,2.0,3.0,2.0,30.0,12.0,25.0,15.0,3.0,13.0,9.0,1.0,4.0,29.0,4.0,30.0,4.0,8.0,1.0,15.0,13.0,9.0,5.0,30.0,20.0,30.0,2.0,5.0,4.0,2.0,1.0,1.0,10.0
3,2.0,2.0,9.0,2.0,1.0,21.0,2.0,2.0,23.0,30.0,30.0,4.0,1.0,1.0,4.0,18.0,23.0,30.0,29.0,9.0,30.0,2.0,24.0,30.0,1.0,2.0,1.0,30.0,10.0,23.0,16.0,2.0,19.0,3.0,1.0,5.0,28.0,6.0,30.0,4.0,19.0,1.0,21.0,23.0,13.0,9.0,30.0,12.0,30.0,3.0,2.0,4.0,1.0,1.0,1.0,11.0
4,5.0,2.0,17.0,1.0,1.0,4.0,7.0,4.0,26.0,30.0,30.0,11.0,7.0,1.0,3.0,24.0,26.0,28.0,29.0,22.0,30.0,2.0,25.0,30.0,1.0,2.0,1.0,30.0,12.0,28.0,20.0,2.0,14.0,1.0,1.0,4.0,25.0,4.0,30.0,7.0,8.0,2.0,21.0,27.0,20.0,4.0,30.0,18.0,30.0,5.0,1.0,4.0,1.0,2.0,2.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,22.0,21.0,12.0,19.0,20.0,12.0,27.0,28.0,9.0,7.0,6.0,18.0,2.0,9.0,15.0,17.0,22.0,3.0,1.0,5.0,4.0,15.0,3.0,2.0,24.0,28.0,27.0,12.0,16.0,6.0,13.0,24.0,27.0,21.0,29.0,24.0,19.0,17.0,7.0,28.0,24.0,6.0,11.0,16.0,22.0,23.0,5.0,1.0,8.0,7.0,21.0,16.0,28.0,18.0,18.0,20.0
55042,22.0,22.0,13.0,20.0,20.0,14.0,27.0,28.0,9.0,6.0,5.0,17.0,2.0,12.0,16.0,18.0,24.0,4.0,1.0,6.0,4.0,16.0,4.0,3.0,21.0,28.0,27.0,14.0,16.0,4.0,14.0,23.0,28.0,20.0,29.0,22.0,17.0,18.0,7.0,28.0,24.0,6.0,11.0,16.0,24.0,24.0,7.0,1.0,7.0,8.0,21.0,16.0,28.0,19.0,19.0,19.0
55043,22.0,21.0,15.0,20.0,19.0,15.0,27.0,28.0,8.0,7.0,6.0,16.0,2.0,13.0,16.0,16.0,23.0,5.0,1.0,9.0,4.0,19.0,4.0,3.0,22.0,28.0,27.0,15.0,14.0,4.0,14.0,23.0,28.0,21.0,28.0,21.0,18.0,17.0,8.0,28.0,24.0,5.0,12.0,16.0,24.0,24.0,7.0,1.0,8.0,9.0,19.0,17.0,28.0,19.0,19.0,19.0
55044,22.0,22.0,15.0,20.0,19.0,13.0,27.0,29.0,8.0,6.0,5.0,15.0,2.0,13.0,15.0,16.0,23.0,4.0,1.0,9.0,5.0,19.0,3.0,2.0,21.0,28.0,27.0,13.0,15.0,5.0,14.0,24.0,28.0,23.0,29.0,23.0,16.0,17.0,6.0,29.0,23.0,6.0,11.0,15.0,23.0,24.0,6.0,1.0,7.0,9.0,22.0,18.0,28.0,19.0,19.0,19.0


In [23]:
run = 1
if run:
    DF_GAME_META = pd.read_csv('./data/2000-2022/facts_boxscores_meta.csv').drop(columns='Unnamed: 0')
    GAME_META = list(DF_GAME_META.columns)
    df_all_seasons_index = pd.read_csv('./data/2000-2022/facts_game_index.csv').drop(columns='Unnamed: 0')
    game_index      = pd.Index(map(tuple,df_all_seasons_index[['game_id','team_id']].values))
    team_prev_game_index    = pd.Index(map(tuple,df_all_seasons_index[['team_prev_game_id','team_id']].values))
    opp_prev_game_index     = pd.Index(map(tuple,df_all_seasons_index[['opp_prev_game_id','opp_id']].values))
    h2h_prev_game_index     = pd.Index(map(tuple,df_all_seasons_index[['h2h_prev_game_id','team_id']].values))
    opp_h2h_prev_game_index = pd.Index(map(tuple,df_all_seasons_index[['h2h_prev_game_id','opp_id']].values))
    
    df_lg_rank = pd.read_csv('./data/2000-2022/facts_cumu_stats_rankings.csv').drop(columns='Unnamed: 0')
    df = df_lg_rank.set_index(game_index)
    
    for team_id in pd.unique(DF_GAME_META['team_id']):
        df.loc[(-1,team_id),:] = np.nan    # Create dummy index/entries

    team_prior      = df.loc[team_prev_game_index].reset_index(drop=True).fillna(0)
    opp_prior       = df.loc[opp_prev_game_index].reset_index(drop=True).fillna(0)
    net_prior       = (team_prior - opp_prior).fillna(0)
    avg_prior       = ((team_prior + opp_prior)/2).fillna(0)
    team_prior.columns = team_prior.columns + '_team'
    opp_prior.columns = opp_prior.columns +  '_opp'
    net_prior.columns = net_prior.columns + '_net'
    avg_prior.columns = avg_prior.columns +  '_avg'
    team_prior.to_csv(f'./data/2000-2022/prior_cumu_stats_rankings_team.csv')
    opp_prior.to_csv(f'./data/2000-2022/prior_cumu_stats_rankings_opp.csv')
    net_prior.to_csv(f'./data/2000-2022/prior_cumu_stats_rankings_net.csv')
    avg_prior.to_csv(f'./data/2000-2022/prior_cumu_stats_rankings_avg.csv')

else:
    team_prior = pd.read_csv(f'./data/2000-2022/prior_cumu_stats_rankings_team.csv').drop(columns='Unnamed: 0')
    opp_prior = pd.read_csv(f'./data/2000-2022/prior_cumu_stats_rankings_opp.csv').drop(columns='Unnamed: 0')
    net_prior = pd.read_csv(f'./data/2000-2022/prior_cumu_stats_rankings_net.csv').drop(columns='Unnamed: 0')
    avg_prior = pd.read_csv(f'./data/2000-2022/prior_cumu_stats_rankings_avg.csv').drop(columns='Unnamed: 0')

print(f"[{dt.datetime.now().strftime('%H:%M:%S %d-%m-%Y')}] cached = {not bool(run)}")
display_html(team_prior)
display_html(opp_prior)
display_html(net_prior)
display_html(avg_prior)

[13:24:12 19-04-2023] cached = False


,win_result_rank_team,pts_rank_team,opp_pts_rank_team,fg_rank_team,fga_rank_team,fg_pct_rank_team,fg3_rank_team,fg3a_rank_team,fg3_pct_rank_team,ft_rank_team,fta_rank_team,ft_pct_rank_team,orb_rank_team,trb_rank_team,ast_rank_team,stl_rank_team,blk_rank_team,tov_rank_team,pf_rank_team,opp_fg_rank_team,opp_fga_rank_team,opp_fg_pct_rank_team,opp_fg3_rank_team,opp_fg3a_rank_team,opp_fg3_pct_rank_team,opp_ft_rank_team,opp_fta_rank_team,opp_ft_pct_rank_team,opp_orb_rank_team,opp_trb_rank_team,opp_ast_rank_team,opp_stl_rank_team,opp_blk_rank_team,opp_tov_rank_team,opp_pf_rank_team,off_rtg_rank_team,def_rtg_rank_team,pace_rank_team,fta_per_fga_pct_rank_team,fg3a_per_fga_pct_rank_team,ts_pct_rank_team,trb_pct_rank_team,ast_pct_rank_team,stl_pct_rank_team,blk_pct_rank_team,efg_pct_rank_team,tov_pct_rank_team,orb_pct_rank_team,ft_rate_rank_team,opp_efg_pct_rank_team,opp_tov_pct_rank_team,drb_pct_rank_team,opp_ft_rate_rank_team,net_pts_rank_team,net_rtg_rank_team,total_pts_rank_team
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,30.0,30.0,1.0,30.0,2.0,1.0,8.0,1.0,25.0,30.0,2.0,29.0,1.0,30.0,30.0,4.0,1.0,1.0,30.0,7.0,28.0,1.0,6.0,1.0,24.0,16.0,5.0,30.0,1.0,30.0,4.0,1.0,1.0,1.0,15.0,1.0,1.0,27.0,30.0,30.0,1.0,19.0,1.0,1.0,1.0,1.0,14.0
2,11.0,10.0,4.0,2.0,1.0,28.0,1.0,2.0,15.0,30.0,30.0,19.0,9.0,1.0,2.0,21.0,8.0,30.0,27.0,2.0,29.0,1.0,24.0,30.0,2.0,3.0,2.0,30.0,12.0,30.0,6.0,2.0,8.0,6.0,1.0,22.0,30.0,4.0,30.0,4.0,28.0,5.0,9.0,27.0,2.0,15.0,30.0,26.0,30.0,1.0,4.0,4.0,2.0,1.0,2.0,16.0
3,4.0,2.0,8.0,2.0,1.0,11.0,1.0,2.0,12.0,30.0,30.0,7.0,15.0,1.0,2.0,7.0,22.0,29.0,27.0,5.0,29.0,1.0,23.0,30.0,2.0,3.0,2.0,30.0,12.0,25.0,15.0,3.0,13.0,9.0,1.0,4.0,29.0,4.0,30.0,4.0,8.0,1.0,15.0,13.0,9.0,5.0,30.0,20.0,30.0,2.0,5.0,4.0,2.0,1.0,1.0,10.0
4,2.0,2.0,9.0,2.0,1.0,21.0,2.0,2.0,23.0,30.0,30.0,4.0,1.0,1.0,4.0,18.0,23.0,30.0,29.0,9.0,30.0,2.0,24.0,30.0,1.0,2.0,1.0,30.0,10.0,23.0,16.0,2.0,19.0,3.0,1.0,5.0,28.0,6.0,30.0,4.0,19.0,1.0,21.0,23.0,13.0,9.0,30.0,12.0,30.0,3.0,2.0,4.0,1.0,1.0,1.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,23.0,21.0,11.0,19.0,20.0,11.0,27.0,29.0,9.0,7.0,7.0,18.0,2.0,11.0,15.0,16.0,22.0,4.0,1.0,6.0,5.0,15.0,3.0,2.0,23.0,28.0,27.0,13.0,16.0,6.0,14.0,24.0,27.0,21.0,29.0,23.0,19.0,17.0,6.0,29.0,23.0,6.0,11.0,16.0,22.0,24.0,6.0,1.0,7.0,7.0,21.0,17.0,28.0,19.0,18.0,20.0
55042,22.0,21.0,12.0,19.0,20.0,12.0,27.0,28.0,9.0,7.0,6.0,18.0,2.0,9.0,15.0,17.0,22.0,3.0,1.0,5.0,4.0,15.0,3.0,2.0,24.0,28.0,27.0,12.0,16.0,6.0,13.0,24.0,27.0,21.0,29.0,24.0,19.0,17.0,7.0,28.0,24.0,6.0,11.0,16.0,22.0,23.0,5.0,1.0,8.0,7.0,21.0,16.0,28.0,18.0,18.0,20.0
55043,22.0,22.0,13.0,20.0,20.0,14.0,27.0,28.0,9.0,6.0,5.0,17.0,2.0,12.0,16.0,18.0,24.0,4.0,1.0,6.0,4.0,16.0,4.0,3.0,21.0,28.0,27.0,14.0,16.0,4.0,14.0,23.0,28.0,20.0,29.0,22.0,17.0,18.0,7.0,28.0,24.0,6.0,11.0,16.0,24.0,24.0,7.0,1.0,7.0,8.0,21.0,16.0,28.0,19.0,19.0,19.0
55044,22.0,21.0,15.0,20.0,19.0,15.0,27.0,28.0,8.0,7.0,6.0,16.0,2.0,13.0,16.0,16.0,23.0,5.0,1.0,9.0,4.0,19.0,4.0,3.0,22.0,28.0,27.0,15.0,14.0,4.0,14.0,23.0,28.0,21.0,28.0,21.0,18.0,17.0,8.0,28.0,24.0,5.0,12.0,16.0,24.0,24.0,7.0,1.0,8.0,9.0,19.0,17.0,28.0,19.0,19.0,19.0


,win_result_rank_opp,pts_rank_opp,opp_pts_rank_opp,fg_rank_opp,fga_rank_opp,fg_pct_rank_opp,fg3_rank_opp,fg3a_rank_opp,fg3_pct_rank_opp,ft_rank_opp,fta_rank_opp,ft_pct_rank_opp,orb_rank_opp,trb_rank_opp,ast_rank_opp,stl_rank_opp,blk_rank_opp,tov_rank_opp,pf_rank_opp,opp_fg_rank_opp,opp_fga_rank_opp,opp_fg_pct_rank_opp,opp_fg3_rank_opp,opp_fg3a_rank_opp,opp_fg3_pct_rank_opp,opp_ft_rank_opp,opp_fta_rank_opp,opp_ft_pct_rank_opp,opp_orb_rank_opp,opp_trb_rank_opp,opp_ast_rank_opp,opp_stl_rank_opp,opp_blk_rank_opp,opp_tov_rank_opp,opp_pf_rank_opp,off_rtg_rank_opp,def_rtg_rank_opp,pace_rank_opp,fta_per_fga_pct_rank_opp,fg3a_per_fga_pct_rank_opp,ts_pct_rank_opp,trb_pct_rank_opp,ast_pct_rank_opp,stl_pct_rank_opp,blk_pct_rank_opp,efg_pct_rank_opp,tov_pct_rank_opp,orb_pct_rank_opp,ft_rate_rank_opp,opp_efg_pct_rank_opp,opp_tov_pct_rank_opp,drb_pct_rank_opp,opp_ft_rate_rank_opp,net_pts_rank_opp,net_rtg_rank_opp,total_pts_rank_opp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.0,5.0,26.0,8.0,10.0,8.0,5.0,3.0,25.0,9.0,5.0,25.0,27.0,8.0,3.0,1.0,19.0,7.0,12.0,24.0,20.0,25.0,23.0,23.0,20.0,7.0,9.0,8.0,1.0,14.0,14.0,26.0,13.0,25.0,27.0,14.0,20.0,3.0,11.0,1.0,3.0,9.0,1.0,3.0,11.0,3.0,9.0,24.0,15.0,26.0,23.0,2.0,9.0,10.0,11.0,5.0
2,19.0,21.0,14.0,21.0,7.0,30.0,14.0,8.0,27.0,19.0,20.0,9.0,23.0,2.0,14.0,28.0,29.0,29.0,6.0,10.0,22.0,4.0,17.0,22.0,1.0,25.0,26.0,9.0,3.0,26.0,1.0,3.0,29.0,8.0,30.0,28.0,27.0,5.0,22.0,8.0,29.0,17.0,6.0,28.0,29.0,29.0,29.0,28.0,22.0,5.0,6.0,1.0,23.0,18.0,19.0,17.0
3,20.0,23.0,12.0,27.0,24.0,23.0,18.0,18.0,6.0,5.0,19.0,1.0,29.0,30.0,13.0,25.0,17.0,27.0,28.0,19.0,17.0,14.0,20.0,18.0,19.0,3.0,3.0,26.0,18.0,20.0,20.0,1.0,18.0,13.0,17.0,21.0,12.0,23.0,13.0,17.0,14.0,27.0,3.0,24.0,15.0,22.0,24.0,28.0,4.0,20.0,15.0,19.0,5.0,22.0,21.0,20.0
4,26.0,13.0,20.0,7.0,6.0,19.0,23.0,26.0,1.0,21.0,23.0,5.0,12.0,25.0,11.0,17.0,15.0,23.0,8.0,25.0,26.0,21.0,3.0,7.0,12.0,22.0,22.0,10.0,30.0,30.0,25.0,11.0,21.0,29.0,24.0,20.0,15.0,10.0,27.0,28.0,25.0,30.0,18.0,19.0,22.0,25.0,26.0,18.0,22.0,9.0,27.0,30.0,16.0,19.0,18.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,22.0,20.0,14.0,22.0,21.0,18.0,26.0,27.0,20.0,1.0,1.0,8.0,9.0,21.0,20.0,25.0,28.0,26.0,8.0,16.0,7.0,23.0,9.0,9.0,4.0,21.0,21.0,26.0,14.0,4.0,22.0,8.0,8.0,8.0,28.0,13.0,8.0,23.0,3.0,27.0,22.0,13.0,14.0,25.0,28.0,26.0,23.0,5.0,1.0,18.0,7.0,16.0,24.0,18.0,19.0,19.0
55042,13.0,2.0,26.0,8.0,11.0,5.0,5.0,2.0,25.0,9.0,5.0,23.0,26.0,14.0,3.0,2.0,16.0,9.0,11.0,26.0,22.0,26.0,23.0,23.0,22.0,6.0,9.0,9.0,1.0,11.0,13.0,25.0,10.0,23.0,25.0,11.0,14.0,3.0,10.0,2.0,5.0,11.0,3.0,4.0,10.0,4.0,9.0,27.0,12.0,26.0,19.0,2.0,7.0,12.0,12.0,5.0
55043,16.0,13.0,18.0,16.0,15.0,17.0,21.0,22.0,6.0,3.0,9.0,6.0,5.0,13.0,21.0,14.0,13.0,15.0,13.0,17.0,15.0,13.0,12.0,13.0,8.0,26.0,25.0,28.0,19.0,16.0,9.0,13.0,25.0,16.0,17.0,12.0,10.0,17.0,10.0,24.0,19.0,14.0,21.0,14.0,18.0,23.0,19.0,7.0,5.0,10.0,14.0,20.0,26.0,16.0,16.0,13.0
55044,29.0,3.0,29.0,2.0,1.0,25.0,1.0,1.0,26.0,24.0,21.0,18.0,8.0,10.0,6.0,2.0,22.0,15.0,14.0,29.0,28.0,29.0,29.0,29.0,29.0,16.0,13.0,6.0,3.0,23.0,29.0,6.0,23.0,22.0,6.0,15.0,1.0,1.0,29.0,1.0,16.0,19.0,15.0,5.0,12.0,12.0,24.0,16.0,28.0,29.0,14.0,4.0,11.0,29.0,29.0,1.0


,win_result_rank_net,pts_rank_net,opp_pts_rank_net,fg_rank_net,fga_rank_net,fg_pct_rank_net,fg3_rank_net,fg3a_rank_net,fg3_pct_rank_net,ft_rank_net,fta_rank_net,ft_pct_rank_net,orb_rank_net,trb_rank_net,ast_rank_net,stl_rank_net,blk_rank_net,tov_rank_net,pf_rank_net,opp_fg_rank_net,opp_fga_rank_net,opp_fg_pct_rank_net,opp_fg3_rank_net,opp_fg3a_rank_net,opp_fg3_pct_rank_net,opp_ft_rank_net,opp_fta_rank_net,opp_ft_pct_rank_net,opp_orb_rank_net,opp_trb_rank_net,opp_ast_rank_net,opp_stl_rank_net,opp_blk_rank_net,opp_tov_rank_net,opp_pf_rank_net,off_rtg_rank_net,def_rtg_rank_net,pace_rank_net,fta_per_fga_pct_rank_net,fg3a_per_fga_pct_rank_net,ts_pct_rank_net,trb_pct_rank_net,ast_pct_rank_net,stl_pct_rank_net,blk_pct_rank_net,efg_pct_rank_net,tov_pct_rank_net,orb_pct_rank_net,ft_rate_rank_net,opp_efg_pct_rank_net,opp_tov_pct_rank_net,drb_pct_rank_net,opp_ft_rate_rank_net,net_pts_rank_net,net_rtg_rank_net,total_pts_rank_net
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-12.0,-3.0,-24.0,-7.0,-9.0,-6.0,-4.0,-2.0,-24.0,21.0,25.0,-24.0,3.0,-6.0,-2.0,7.0,-18.0,18.0,18.0,-22.0,9.0,-24.0,7.0,7.0,-16.0,-6.0,-8.0,22.0,6.0,14.0,-13.0,-20.0,-12.0,-1.0,-11.0,-9.0,10.0,-2.0,19.0,3.0,-2.0,-8.0,0.0,12.0,-10.0,-2.0,18.0,6.0,15.0,-25.0,-4.0,-1.0,-8.0,-9.0,-10.0,9.0
2,-8.0,-11.0,-10.0,-19.0,-6.0,-2.0,-13.0,-6.0,-12.0,11.0,10.0,10.0,-14.0,-1.0,-12.0,-7.0,-21.0,1.0,21.0,-8.0,7.0,-3.0,7.0,8.0,1.0,-22.0,-24.0,21.0,9.0,4.0,5.0,-1.0,-21.0,-2.0,-29.0,-6.0,3.0,-1.0,8.0,-4.0,-1.0,-12.0,3.0,-1.0,-27.0,-14.0,1.0,-2.0,8.0,-4.0,-2.0,3.0,-21.0,-17.0,-17.0,-1.0
3,-16.0,-21.0,-4.0,-25.0,-23.0,-12.0,-17.0,-16.0,6.0,25.0,11.0,6.0,-14.0,-29.0,-11.0,-18.0,5.0,2.0,-1.0,-14.0,12.0,-13.0,3.0,12.0,-17.0,0.0,-1.0,4.0,-6.0,5.0,-5.0,2.0,-5.0,-4.0,-16.0,-17.0,17.0,-19.0,17.0,-13.0,-6.0,-26.0,12.0,-11.0,-6.0,-17.0,6.0,-8.0,26.0,-18.0,-10.0,-15.0,-3.0,-21.0,-20.0,-10.0
4,-24.0,-11.0,-11.0,-5.0,-5.0,2.0,-21.0,-24.0,22.0,9.0,7.0,-1.0,-11.0,-24.0,-7.0,1.0,8.0,7.0,21.0,-16.0,4.0,-19.0,21.0,23.0,-11.0,-20.0,-21.0,20.0,-20.0,-7.0,-9.0,-9.0,-2.0,-26.0,-23.0,-15.0,13.0,-4.0,3.0,-24.0,-6.0,-29.0,3.0,4.0,-9.0,-16.0,4.0,-6.0,8.0,-6.0,-25.0,-26.0,-15.0,-18.0,-17.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,1.0,1.0,-3.0,-3.0,-1.0,-7.0,1.0,2.0,-11.0,6.0,6.0,10.0,-7.0,-10.0,-5.0,-9.0,-6.0,-22.0,-7.0,-10.0,-2.0,-8.0,-6.0,-7.0,19.0,7.0,6.0,-13.0,2.0,2.0,-8.0,16.0,19.0,13.0,1.0,10.0,11.0,-6.0,3.0,2.0,1.0,-7.0,-3.0,-9.0,-6.0,-2.0,-17.0,-4.0,6.0,-11.0,14.0,1.0,4.0,1.0,-1.0,1.0
55042,9.0,19.0,-14.0,11.0,9.0,7.0,22.0,26.0,-16.0,-2.0,1.0,-5.0,-24.0,-5.0,12.0,15.0,6.0,-6.0,-10.0,-21.0,-18.0,-11.0,-20.0,-21.0,2.0,22.0,18.0,3.0,15.0,-5.0,0.0,-1.0,17.0,-2.0,4.0,13.0,5.0,14.0,-3.0,26.0,19.0,-5.0,8.0,12.0,12.0,19.0,-4.0,-26.0,-4.0,-19.0,2.0,14.0,21.0,6.0,6.0,15.0
55043,6.0,9.0,-5.0,4.0,5.0,-3.0,6.0,6.0,3.0,3.0,-4.0,11.0,-3.0,-1.0,-5.0,4.0,11.0,-11.0,-12.0,-11.0,-11.0,3.0,-8.0,-10.0,13.0,2.0,2.0,-14.0,-3.0,-12.0,5.0,10.0,3.0,4.0,12.0,10.0,7.0,1.0,-3.0,4.0,5.0,-8.0,-10.0,2.0,6.0,1.0,-12.0,-6.0,2.0,-2.0,7.0,-4.0,2.0,3.0,3.0,6.0
55044,-7.0,18.0,-14.0,18.0,18.0,-10.0,26.0,27.0,-18.0,-17.0,-15.0,-2.0,-6.0,3.0,10.0,14.0,1.0,-10.0,-13.0,-20.0,-24.0,-10.0,-25.0,-26.0,-7.0,12.0,14.0,9.0,11.0,-19.0,-15.0,17.0,5.0,-1.0,22.0,6.0,17.0,16.0,-21.0,27.0,8.0,-14.0,-3.0,11.0,12.0,12.0,-17.0,-15.0,-20.0,-20.0,5.0,13.0,17.0,-10.0,-10.0,18.0


,win_result_rank_avg,pts_rank_avg,opp_pts_rank_avg,fg_rank_avg,fga_rank_avg,fg_pct_rank_avg,fg3_rank_avg,fg3a_rank_avg,fg3_pct_rank_avg,ft_rank_avg,fta_rank_avg,ft_pct_rank_avg,orb_rank_avg,trb_rank_avg,ast_rank_avg,stl_rank_avg,blk_rank_avg,tov_rank_avg,pf_rank_avg,opp_fg_rank_avg,opp_fga_rank_avg,opp_fg_pct_rank_avg,opp_fg3_rank_avg,opp_fg3a_rank_avg,opp_fg3_pct_rank_avg,opp_ft_rank_avg,opp_fta_rank_avg,opp_ft_pct_rank_avg,opp_orb_rank_avg,opp_trb_rank_avg,opp_ast_rank_avg,opp_stl_rank_avg,opp_blk_rank_avg,opp_tov_rank_avg,opp_pf_rank_avg,off_rtg_rank_avg,def_rtg_rank_avg,pace_rank_avg,fta_per_fga_pct_rank_avg,fg3a_per_fga_pct_rank_avg,ts_pct_rank_avg,trb_pct_rank_avg,ast_pct_rank_avg,stl_pct_rank_avg,blk_pct_rank_avg,efg_pct_rank_avg,tov_pct_rank_avg,orb_pct_rank_avg,ft_rate_rank_avg,opp_efg_pct_rank_avg,opp_tov_pct_rank_avg,drb_pct_rank_avg,opp_ft_rate_rank_avg,net_pts_rank_avg,net_rtg_rank_avg,total_pts_rank_avg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.0,3.5,14.0,4.5,5.5,5.0,3.0,2.0,13.0,19.5,17.5,13.0,28.5,5.0,2.0,4.5,10.0,16.0,21.0,13.0,24.5,13.0,26.5,26.5,12.0,4.0,5.0,19.0,4.0,21.0,7.5,16.0,7.0,24.5,21.5,9.5,25.0,2.0,20.5,2.5,2.0,5.0,1.0,9.0,6.0,2.0,18.0,27.0,22.5,13.5,21.0,1.5,5.0,5.5,6.0,9.5
2,15.0,15.5,9.0,11.5,4.0,29.0,7.5,5.0,21.0,24.5,25.0,14.0,16.0,1.5,8.0,24.5,18.5,29.5,16.5,6.0,25.5,2.5,20.5,26.0,1.5,14.0,14.0,19.5,7.5,28.0,3.5,2.5,18.5,7.0,15.5,25.0,28.5,4.5,26.0,6.0,28.5,11.0,7.5,27.5,15.5,22.0,29.5,27.0,26.0,3.0,5.0,2.5,12.5,9.5,10.5,16.5
3,12.0,12.5,10.0,14.5,12.5,17.0,9.5,10.0,9.0,17.5,24.5,4.0,22.0,15.5,7.5,16.0,19.5,28.0,27.5,12.0,23.0,7.5,21.5,24.0,10.5,3.0,2.5,28.0,15.0,22.5,17.5,2.0,15.5,11.0,9.0,12.5,20.5,13.5,21.5,10.5,11.0,14.0,9.0,18.5,12.0,13.5,27.0,24.0,17.0,11.0,10.0,11.5,3.5,11.5,11.0,15.0
4,14.0,7.5,14.5,4.5,3.5,20.0,12.5,14.0,12.0,25.5,26.5,4.5,6.5,13.0,7.5,17.5,19.0,26.5,18.5,17.0,28.0,11.5,13.5,18.5,6.5,12.0,11.5,20.0,20.0,26.5,20.5,6.5,20.0,16.0,12.5,12.5,21.5,8.0,28.5,16.0,22.0,15.5,19.5,21.0,17.5,17.0,28.0,15.0,26.0,6.0,14.5,17.0,8.5,10.0,9.5,11.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,22.5,20.5,12.5,20.5,20.5,14.5,26.5,28.0,14.5,4.0,4.0,13.0,5.5,16.0,17.5,20.5,25.0,15.0,4.5,11.0,6.0,19.0,6.0,5.5,13.5,24.5,24.0,19.5,15.0,5.0,18.0,16.0,17.5,14.5,28.5,18.0,13.5,20.0,4.5,28.0,22.5,9.5,12.5,20.5,25.0,25.0,14.5,3.0,4.0,12.5,14.0,16.5,26.0,18.5,18.5,19.5
55042,17.5,11.5,19.0,13.5,15.5,8.5,16.0,15.0,17.0,8.0,5.5,20.5,14.0,11.5,9.0,9.5,19.0,6.0,6.0,15.5,13.0,20.5,13.0,12.5,23.0,17.0,18.0,10.5,8.5,8.5,13.0,24.5,18.5,22.0,27.0,17.5,16.5,10.0,8.5,15.0,14.5,8.5,7.0,10.0,16.0,13.5,7.0,14.0,10.0,16.5,20.0,9.0,17.5,15.0,15.0,12.5
55043,19.0,17.5,15.5,18.0,17.5,15.5,24.0,25.0,7.5,4.5,7.0,11.5,3.5,12.5,18.5,16.0,18.5,9.5,7.0,11.5,9.5,14.5,8.0,8.0,14.5,27.0,26.0,21.0,17.5,10.0,11.5,18.0,26.5,18.0,23.0,17.0,13.5,17.5,8.5,26.0,21.5,10.0,16.0,15.0,21.0,23.5,13.0,4.0,6.0,9.0,17.5,18.0,27.0,17.5,17.5,16.0
55044,25.5,12.0,22.0,11.0,10.0,20.0,14.0,14.5,17.0,15.5,13.5,17.0,5.0,11.5,11.0,9.0,22.5,10.0,7.5,19.0,16.0,24.0,16.5,16.0,25.5,22.0,20.0,10.5,8.5,13.5,21.5,14.5,25.5,21.5,17.0,18.0,9.5,9.0,18.5,14.5,20.0,12.0,13.5,10.5,18.0,18.0,15.5,8.5,18.0,19.0,16.5,10.5,19.5,24.0,24.0,10.0


Home record, Away record, Conference record, Cross conference record, Head 2 Head record

April 17: Add scheduling and record stats

In [27]:
run = 1
if run:
    DF_GAME_META = pd.read_csv('./data/2000-2022/facts_boxscores_meta.csv').drop(columns='Unnamed: 0')
    DF_GAME_STATS = pd.read_csv('./data/2000-2022/facts_boxscores_stats.csv').drop(columns='Unnamed: 0')
    GAME_META = list(DF_GAME_META.columns)

    df_records = pd.concat([DF_GAME_META,DF_GAME_STATS['win_result']],axis=1)
    for season in tqdm(pd.unique(df_records['season'])):
        for team_id in pd.unique(df_records[(df_records['season'] == season)]['team_id']):
            filters = (df_records['season'] == season) & (df_records['team_id'] == team_id)
            df_season_team = df_records.loc[filters]

            home_win_pct = (df_season_team['game_location'].astype(bool) & df_season_team['win_result'].astype(bool)).expanding().sum()\
                                                                        /df_season_team['game_location'].astype(bool).expanding().sum()
            road_win_pct = (~df_season_team['game_location'].astype(bool) & df_season_team['win_result'].astype(bool)).expanding().sum()\
                                                                        /(~df_season_team['game_location'].astype(bool)).expanding().sum()
            same_conf_win_pct = (df_season_team['same_conf'].astype(bool) & df_season_team['win_result'].astype(bool)).expanding().sum()\
                                                                        /df_season_team['same_conf'].astype(bool).expanding().sum()
            cross_conf_win_pct = (~df_season_team['same_conf'].astype(bool) & df_season_team['win_result'].astype(bool)).expanding().sum()\
                                                                        /(~df_season_team['same_conf'].astype(bool)).expanding().sum()
            
            df_records.loc[filters,'home_win_pct'] = home_win_pct.fillna(0)
            df_records.loc[filters,'road_win_pct'] = road_win_pct.fillna(0)
            df_records.loc[filters,'loc_context_win_pct'] = df_records.loc[filters,'home_win_pct'] * df_records.loc[filters,'game_location'] + \
                                                        df_records.loc[filters,'road_win_pct'] * (~df_records.loc[filters,'game_location'].astype(bool)).astype(int)

            df_records.loc[filters,'same_conf_win_pct'] = same_conf_win_pct.fillna(0)
            df_records.loc[filters,'cross_conf_win_pct'] = cross_conf_win_pct.fillna(0)
            df_records.loc[filters,'conf_context_win_pct'] = df_records.loc[filters,'same_conf_win_pct'] * df_records.loc[filters,'same_conf'] + \
                                                    df_records.loc[filters,'cross_conf_win_pct'] * (~df_records.loc[filters,'same_conf'].astype(bool)).astype(int)

            for opp_id in pd.unique(df_season_team['opp_id']):
                df_series_opp_id = df_season_team[df_season_team['opp_id'] == opp_id].sort_values('date_game',ascending=True)
                df_records.loc[df_series_opp_id.index,'season_h2h_win_pct'] = df_series_opp_id['win_result'].expanding().mean().fillna(0.5)

    df_records =  df_records.drop(columns=GAME_META + ['win_result'])
    df_records.to_csv('./data/2000-2022/facts_cumu_win_records.csv')
else:
    df_records = pd.read_csv('./data/2000-2022/facts_cumu_win_records.csv').drop(columns='Unnamed: 0')

print(f"[{dt.datetime.now().strftime('%H:%M:%S %d-%m-%Y')}] cached = {not bool(run)}")
df_records

100%|██████████| 23/23 [00:11<00:00,  1.95it/s]


[13:43:37 19-04-2023] cached = False


,win_result_rank_team,pts_rank_team,opp_pts_rank_team,fg_rank_team,fga_rank_team,fg_pct_rank_team,fg3_rank_team,fg3a_rank_team,fg3_pct_rank_team,ft_rank_team,fta_rank_team,ft_pct_rank_team,orb_rank_team,trb_rank_team,ast_rank_team,stl_rank_team,blk_rank_team,tov_rank_team,pf_rank_team,opp_fg_rank_team,opp_fga_rank_team,opp_fg_pct_rank_team,opp_fg3_rank_team,opp_fg3a_rank_team,opp_fg3_pct_rank_team,opp_ft_rank_team,opp_fta_rank_team,opp_ft_pct_rank_team,opp_orb_rank_team,opp_trb_rank_team,opp_ast_rank_team,opp_stl_rank_team,opp_blk_rank_team,opp_tov_rank_team,opp_pf_rank_team,off_rtg_rank_team,def_rtg_rank_team,pace_rank_team,fta_per_fga_pct_rank_team,fg3a_per_fga_pct_rank_team,ts_pct_rank_team,trb_pct_rank_team,ast_pct_rank_team,stl_pct_rank_team,blk_pct_rank_team,efg_pct_rank_team,tov_pct_rank_team,orb_pct_rank_team,ft_rate_rank_team,opp_efg_pct_rank_team,opp_tov_pct_rank_team,drb_pct_rank_team,opp_ft_rate_rank_team,net_pts_rank_team,net_rtg_rank_team,total_pts_rank_team
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,30.0,30.0,1.0,30.0,2.0,1.0,8.0,1.0,25.0,30.0,2.0,29.0,1.0,30.0,30.0,4.0,1.0,1.0,30.0,7.0,28.0,1.0,6.0,1.0,24.0,16.0,5.0,30.0,1.0,30.0,4.0,1.0,1.0,1.0,15.0,1.0,1.0,27.0,30.0,30.0,1.0,19.0,1.0,1.0,1.0,1.0,14.0
2,11.0,10.0,4.0,2.0,1.0,28.0,1.0,2.0,15.0,30.0,30.0,19.0,9.0,1.0,2.0,21.0,8.0,30.0,27.0,2.0,29.0,1.0,24.0,30.0,2.0,3.0,2.0,30.0,12.0,30.0,6.0,2.0,8.0,6.0,1.0,22.0,30.0,4.0,30.0,4.0,28.0,5.0,9.0,27.0,2.0,15.0,30.0,26.0,30.0,1.0,4.0,4.0,2.0,1.0,2.0,16.0
3,4.0,2.0,8.0,2.0,1.0,11.0,1.0,2.0,12.0,30.0,30.0,7.0,15.0,1.0,2.0,7.0,22.0,29.0,27.0,5.0,29.0,1.0,23.0,30.0,2.0,3.0,2.0,30.0,12.0,25.0,15.0,3.0,13.0,9.0,1.0,4.0,29.0,4.0,30.0,4.0,8.0,1.0,15.0,13.0,9.0,5.0,30.0,20.0,30.0,2.0,5.0,4.0,2.0,1.0,1.0,10.0
4,2.0,2.0,9.0,2.0,1.0,21.0,2.0,2.0,23.0,30.0,30.0,4.0,1.0,1.0,4.0,18.0,23.0,30.0,29.0,9.0,30.0,2.0,24.0,30.0,1.0,2.0,1.0,30.0,10.0,23.0,16.0,2.0,19.0,3.0,1.0,5.0,28.0,6.0,30.0,4.0,19.0,1.0,21.0,23.0,13.0,9.0,30.0,12.0,30.0,3.0,2.0,4.0,1.0,1.0,1.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,23.0,21.0,11.0,19.0,20.0,11.0,27.0,29.0,9.0,7.0,7.0,18.0,2.0,11.0,15.0,16.0,22.0,4.0,1.0,6.0,5.0,15.0,3.0,2.0,23.0,28.0,27.0,13.0,16.0,6.0,14.0,24.0,27.0,21.0,29.0,23.0,19.0,17.0,6.0,29.0,23.0,6.0,11.0,16.0,22.0,24.0,6.0,1.0,7.0,7.0,21.0,17.0,28.0,19.0,18.0,20.0
55042,22.0,21.0,12.0,19.0,20.0,12.0,27.0,28.0,9.0,7.0,6.0,18.0,2.0,9.0,15.0,17.0,22.0,3.0,1.0,5.0,4.0,15.0,3.0,2.0,24.0,28.0,27.0,12.0,16.0,6.0,13.0,24.0,27.0,21.0,29.0,24.0,19.0,17.0,7.0,28.0,24.0,6.0,11.0,16.0,22.0,23.0,5.0,1.0,8.0,7.0,21.0,16.0,28.0,18.0,18.0,20.0
55043,22.0,22.0,13.0,20.0,20.0,14.0,27.0,28.0,9.0,6.0,5.0,17.0,2.0,12.0,16.0,18.0,24.0,4.0,1.0,6.0,4.0,16.0,4.0,3.0,21.0,28.0,27.0,14.0,16.0,4.0,14.0,23.0,28.0,20.0,29.0,22.0,17.0,18.0,7.0,28.0,24.0,6.0,11.0,16.0,24.0,24.0,7.0,1.0,7.0,8.0,21.0,16.0,28.0,19.0,19.0,19.0
55044,22.0,21.0,15.0,20.0,19.0,15.0,27.0,28.0,8.0,7.0,6.0,16.0,2.0,13.0,16.0,16.0,23.0,5.0,1.0,9.0,4.0,19.0,4.0,3.0,22.0,28.0,27.0,15.0,14.0,4.0,14.0,23.0,28.0,21.0,28.0,21.0,18.0,17.0,8.0,28.0,24.0,5.0,12.0,16.0,24.0,24.0,7.0,1.0,8.0,9.0,19.0,17.0,28.0,19.0,19.0,19.0


,win_result_rank_opp,pts_rank_opp,opp_pts_rank_opp,fg_rank_opp,fga_rank_opp,fg_pct_rank_opp,fg3_rank_opp,fg3a_rank_opp,fg3_pct_rank_opp,ft_rank_opp,fta_rank_opp,ft_pct_rank_opp,orb_rank_opp,trb_rank_opp,ast_rank_opp,stl_rank_opp,blk_rank_opp,tov_rank_opp,pf_rank_opp,opp_fg_rank_opp,opp_fga_rank_opp,opp_fg_pct_rank_opp,opp_fg3_rank_opp,opp_fg3a_rank_opp,opp_fg3_pct_rank_opp,opp_ft_rank_opp,opp_fta_rank_opp,opp_ft_pct_rank_opp,opp_orb_rank_opp,opp_trb_rank_opp,opp_ast_rank_opp,opp_stl_rank_opp,opp_blk_rank_opp,opp_tov_rank_opp,opp_pf_rank_opp,off_rtg_rank_opp,def_rtg_rank_opp,pace_rank_opp,fta_per_fga_pct_rank_opp,fg3a_per_fga_pct_rank_opp,ts_pct_rank_opp,trb_pct_rank_opp,ast_pct_rank_opp,stl_pct_rank_opp,blk_pct_rank_opp,efg_pct_rank_opp,tov_pct_rank_opp,orb_pct_rank_opp,ft_rate_rank_opp,opp_efg_pct_rank_opp,opp_tov_pct_rank_opp,drb_pct_rank_opp,opp_ft_rate_rank_opp,net_pts_rank_opp,net_rtg_rank_opp,total_pts_rank_opp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.0,5.0,26.0,8.0,10.0,8.0,5.0,3.0,25.0,9.0,5.0,25.0,27.0,8.0,3.0,1.0,19.0,7.0,12.0,24.0,20.0,25.0,23.0,23.0,20.0,7.0,9.0,8.0,1.0,14.0,14.0,26.0,13.0,25.0,27.0,14.0,20.0,3.0,11.0,1.0,3.0,9.0,1.0,3.0,11.0,3.0,9.0,24.0,15.0,26.0,23.0,2.0,9.0,10.0,11.0,5.0
2,19.0,21.0,14.0,21.0,7.0,30.0,14.0,8.0,27.0,19.0,20.0,9.0,23.0,2.0,14.0,28.0,29.0,29.0,6.0,10.0,22.0,4.0,17.0,22.0,1.0,25.0,26.0,9.0,3.0,26.0,1.0,3.0,29.0,8.0,30.0,28.0,27.0,5.0,22.0,8.0,29.0,17.0,6.0,28.0,29.0,29.0,29.0,28.0,22.0,5.0,6.0,1.0,23.0,18.0,19.0,17.0
3,20.0,23.0,12.0,27.0,24.0,23.0,18.0,18.0,6.0,5.0,19.0,1.0,29.0,30.0,13.0,25.0,17.0,27.0,28.0,19.0,17.0,14.0,20.0,18.0,19.0,3.0,3.0,26.0,18.0,20.0,20.0,1.0,18.0,13.0,17.0,21.0,12.0,23.0,13.0,17.0,14.0,27.0,3.0,24.0,15.0,22.0,24.0,28.0,4.0,20.0,15.0,19.0,5.0,22.0,21.0,20.0
4,26.0,13.0,20.0,7.0,6.0,19.0,23.0,26.0,1.0,21.0,23.0,5.0,12.0,25.0,11.0,17.0,15.0,23.0,8.0,25.0,26.0,21.0,3.0,7.0,12.0,22.0,22.0,10.0,30.0,30.0,25.0,11.0,21.0,29.0,24.0,20.0,15.0,10.0,27.0,28.0,25.0,30.0,18.0,19.0,22.0,25.0,26.0,18.0,22.0,9.0,27.0,30.0,16.0,19.0,18.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,22.0,20.0,14.0,22.0,21.0,18.0,26.0,27.0,20.0,1.0,1.0,8.0,9.0,21.0,20.0,25.0,28.0,26.0,8.0,16.0,7.0,23.0,9.0,9.0,4.0,21.0,21.0,26.0,14.0,4.0,22.0,8.0,8.0,8.0,28.0,13.0,8.0,23.0,3.0,27.0,22.0,13.0,14.0,25.0,28.0,26.0,23.0,5.0,1.0,18.0,7.0,16.0,24.0,18.0,19.0,19.0
55042,13.0,2.0,26.0,8.0,11.0,5.0,5.0,2.0,25.0,9.0,5.0,23.0,26.0,14.0,3.0,2.0,16.0,9.0,11.0,26.0,22.0,26.0,23.0,23.0,22.0,6.0,9.0,9.0,1.0,11.0,13.0,25.0,10.0,23.0,25.0,11.0,14.0,3.0,10.0,2.0,5.0,11.0,3.0,4.0,10.0,4.0,9.0,27.0,12.0,26.0,19.0,2.0,7.0,12.0,12.0,5.0
55043,16.0,13.0,18.0,16.0,15.0,17.0,21.0,22.0,6.0,3.0,9.0,6.0,5.0,13.0,21.0,14.0,13.0,15.0,13.0,17.0,15.0,13.0,12.0,13.0,8.0,26.0,25.0,28.0,19.0,16.0,9.0,13.0,25.0,16.0,17.0,12.0,10.0,17.0,10.0,24.0,19.0,14.0,21.0,14.0,18.0,23.0,19.0,7.0,5.0,10.0,14.0,20.0,26.0,16.0,16.0,13.0
55044,29.0,3.0,29.0,2.0,1.0,25.0,1.0,1.0,26.0,24.0,21.0,18.0,8.0,10.0,6.0,2.0,22.0,15.0,14.0,29.0,28.0,29.0,29.0,29.0,29.0,16.0,13.0,6.0,3.0,23.0,29.0,6.0,23.0,22.0,6.0,15.0,1.0,1.0,29.0,1.0,16.0,19.0,15.0,5.0,12.0,12.0,24.0,16.0,28.0,29.0,14.0,4.0,11.0,29.0,29.0,1.0


,win_result_rank_net,pts_rank_net,opp_pts_rank_net,fg_rank_net,fga_rank_net,fg_pct_rank_net,fg3_rank_net,fg3a_rank_net,fg3_pct_rank_net,ft_rank_net,fta_rank_net,ft_pct_rank_net,orb_rank_net,trb_rank_net,ast_rank_net,stl_rank_net,blk_rank_net,tov_rank_net,pf_rank_net,opp_fg_rank_net,opp_fga_rank_net,opp_fg_pct_rank_net,opp_fg3_rank_net,opp_fg3a_rank_net,opp_fg3_pct_rank_net,opp_ft_rank_net,opp_fta_rank_net,opp_ft_pct_rank_net,opp_orb_rank_net,opp_trb_rank_net,opp_ast_rank_net,opp_stl_rank_net,opp_blk_rank_net,opp_tov_rank_net,opp_pf_rank_net,off_rtg_rank_net,def_rtg_rank_net,pace_rank_net,fta_per_fga_pct_rank_net,fg3a_per_fga_pct_rank_net,ts_pct_rank_net,trb_pct_rank_net,ast_pct_rank_net,stl_pct_rank_net,blk_pct_rank_net,efg_pct_rank_net,tov_pct_rank_net,orb_pct_rank_net,ft_rate_rank_net,opp_efg_pct_rank_net,opp_tov_pct_rank_net,drb_pct_rank_net,opp_ft_rate_rank_net,net_pts_rank_net,net_rtg_rank_net,total_pts_rank_net
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-12.0,-3.0,-24.0,-7.0,-9.0,-6.0,-4.0,-2.0,-24.0,21.0,25.0,-24.0,3.0,-6.0,-2.0,7.0,-18.0,18.0,18.0,-22.0,9.0,-24.0,7.0,7.0,-16.0,-6.0,-8.0,22.0,6.0,14.0,-13.0,-20.0,-12.0,-1.0,-11.0,-9.0,10.0,-2.0,19.0,3.0,-2.0,-8.0,0.0,12.0,-10.0,-2.0,18.0,6.0,15.0,-25.0,-4.0,-1.0,-8.0,-9.0,-10.0,9.0
2,-8.0,-11.0,-10.0,-19.0,-6.0,-2.0,-13.0,-6.0,-12.0,11.0,10.0,10.0,-14.0,-1.0,-12.0,-7.0,-21.0,1.0,21.0,-8.0,7.0,-3.0,7.0,8.0,1.0,-22.0,-24.0,21.0,9.0,4.0,5.0,-1.0,-21.0,-2.0,-29.0,-6.0,3.0,-1.0,8.0,-4.0,-1.0,-12.0,3.0,-1.0,-27.0,-14.0,1.0,-2.0,8.0,-4.0,-2.0,3.0,-21.0,-17.0,-17.0,-1.0
3,-16.0,-21.0,-4.0,-25.0,-23.0,-12.0,-17.0,-16.0,6.0,25.0,11.0,6.0,-14.0,-29.0,-11.0,-18.0,5.0,2.0,-1.0,-14.0,12.0,-13.0,3.0,12.0,-17.0,0.0,-1.0,4.0,-6.0,5.0,-5.0,2.0,-5.0,-4.0,-16.0,-17.0,17.0,-19.0,17.0,-13.0,-6.0,-26.0,12.0,-11.0,-6.0,-17.0,6.0,-8.0,26.0,-18.0,-10.0,-15.0,-3.0,-21.0,-20.0,-10.0
4,-24.0,-11.0,-11.0,-5.0,-5.0,2.0,-21.0,-24.0,22.0,9.0,7.0,-1.0,-11.0,-24.0,-7.0,1.0,8.0,7.0,21.0,-16.0,4.0,-19.0,21.0,23.0,-11.0,-20.0,-21.0,20.0,-20.0,-7.0,-9.0,-9.0,-2.0,-26.0,-23.0,-15.0,13.0,-4.0,3.0,-24.0,-6.0,-29.0,3.0,4.0,-9.0,-16.0,4.0,-6.0,8.0,-6.0,-25.0,-26.0,-15.0,-18.0,-17.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,1.0,1.0,-3.0,-3.0,-1.0,-7.0,1.0,2.0,-11.0,6.0,6.0,10.0,-7.0,-10.0,-5.0,-9.0,-6.0,-22.0,-7.0,-10.0,-2.0,-8.0,-6.0,-7.0,19.0,7.0,6.0,-13.0,2.0,2.0,-8.0,16.0,19.0,13.0,1.0,10.0,11.0,-6.0,3.0,2.0,1.0,-7.0,-3.0,-9.0,-6.0,-2.0,-17.0,-4.0,6.0,-11.0,14.0,1.0,4.0,1.0,-1.0,1.0
55042,9.0,19.0,-14.0,11.0,9.0,7.0,22.0,26.0,-16.0,-2.0,1.0,-5.0,-24.0,-5.0,12.0,15.0,6.0,-6.0,-10.0,-21.0,-18.0,-11.0,-20.0,-21.0,2.0,22.0,18.0,3.0,15.0,-5.0,0.0,-1.0,17.0,-2.0,4.0,13.0,5.0,14.0,-3.0,26.0,19.0,-5.0,8.0,12.0,12.0,19.0,-4.0,-26.0,-4.0,-19.0,2.0,14.0,21.0,6.0,6.0,15.0
55043,6.0,9.0,-5.0,4.0,5.0,-3.0,6.0,6.0,3.0,3.0,-4.0,11.0,-3.0,-1.0,-5.0,4.0,11.0,-11.0,-12.0,-11.0,-11.0,3.0,-8.0,-10.0,13.0,2.0,2.0,-14.0,-3.0,-12.0,5.0,10.0,3.0,4.0,12.0,10.0,7.0,1.0,-3.0,4.0,5.0,-8.0,-10.0,2.0,6.0,1.0,-12.0,-6.0,2.0,-2.0,7.0,-4.0,2.0,3.0,3.0,6.0
55044,-7.0,18.0,-14.0,18.0,18.0,-10.0,26.0,27.0,-18.0,-17.0,-15.0,-2.0,-6.0,3.0,10.0,14.0,1.0,-10.0,-13.0,-20.0,-24.0,-10.0,-25.0,-26.0,-7.0,12.0,14.0,9.0,11.0,-19.0,-15.0,17.0,5.0,-1.0,22.0,6.0,17.0,16.0,-21.0,27.0,8.0,-14.0,-3.0,11.0,12.0,12.0,-17.0,-15.0,-20.0,-20.0,5.0,13.0,17.0,-10.0,-10.0,18.0


,win_result_rank_avg,pts_rank_avg,opp_pts_rank_avg,fg_rank_avg,fga_rank_avg,fg_pct_rank_avg,fg3_rank_avg,fg3a_rank_avg,fg3_pct_rank_avg,ft_rank_avg,fta_rank_avg,ft_pct_rank_avg,orb_rank_avg,trb_rank_avg,ast_rank_avg,stl_rank_avg,blk_rank_avg,tov_rank_avg,pf_rank_avg,opp_fg_rank_avg,opp_fga_rank_avg,opp_fg_pct_rank_avg,opp_fg3_rank_avg,opp_fg3a_rank_avg,opp_fg3_pct_rank_avg,opp_ft_rank_avg,opp_fta_rank_avg,opp_ft_pct_rank_avg,opp_orb_rank_avg,opp_trb_rank_avg,opp_ast_rank_avg,opp_stl_rank_avg,opp_blk_rank_avg,opp_tov_rank_avg,opp_pf_rank_avg,off_rtg_rank_avg,def_rtg_rank_avg,pace_rank_avg,fta_per_fga_pct_rank_avg,fg3a_per_fga_pct_rank_avg,ts_pct_rank_avg,trb_pct_rank_avg,ast_pct_rank_avg,stl_pct_rank_avg,blk_pct_rank_avg,efg_pct_rank_avg,tov_pct_rank_avg,orb_pct_rank_avg,ft_rate_rank_avg,opp_efg_pct_rank_avg,opp_tov_pct_rank_avg,drb_pct_rank_avg,opp_ft_rate_rank_avg,net_pts_rank_avg,net_rtg_rank_avg,total_pts_rank_avg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.0,3.5,14.0,4.5,5.5,5.0,3.0,2.0,13.0,19.5,17.5,13.0,28.5,5.0,2.0,4.5,10.0,16.0,21.0,13.0,24.5,13.0,26.5,26.5,12.0,4.0,5.0,19.0,4.0,21.0,7.5,16.0,7.0,24.5,21.5,9.5,25.0,2.0,20.5,2.5,2.0,5.0,1.0,9.0,6.0,2.0,18.0,27.0,22.5,13.5,21.0,1.5,5.0,5.5,6.0,9.5
2,15.0,15.5,9.0,11.5,4.0,29.0,7.5,5.0,21.0,24.5,25.0,14.0,16.0,1.5,8.0,24.5,18.5,29.5,16.5,6.0,25.5,2.5,20.5,26.0,1.5,14.0,14.0,19.5,7.5,28.0,3.5,2.5,18.5,7.0,15.5,25.0,28.5,4.5,26.0,6.0,28.5,11.0,7.5,27.5,15.5,22.0,29.5,27.0,26.0,3.0,5.0,2.5,12.5,9.5,10.5,16.5
3,12.0,12.5,10.0,14.5,12.5,17.0,9.5,10.0,9.0,17.5,24.5,4.0,22.0,15.5,7.5,16.0,19.5,28.0,27.5,12.0,23.0,7.5,21.5,24.0,10.5,3.0,2.5,28.0,15.0,22.5,17.5,2.0,15.5,11.0,9.0,12.5,20.5,13.5,21.5,10.5,11.0,14.0,9.0,18.5,12.0,13.5,27.0,24.0,17.0,11.0,10.0,11.5,3.5,11.5,11.0,15.0
4,14.0,7.5,14.5,4.5,3.5,20.0,12.5,14.0,12.0,25.5,26.5,4.5,6.5,13.0,7.5,17.5,19.0,26.5,18.5,17.0,28.0,11.5,13.5,18.5,6.5,12.0,11.5,20.0,20.0,26.5,20.5,6.5,20.0,16.0,12.5,12.5,21.5,8.0,28.5,16.0,22.0,15.5,19.5,21.0,17.5,17.0,28.0,15.0,26.0,6.0,14.5,17.0,8.5,10.0,9.5,11.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55041,22.5,20.5,12.5,20.5,20.5,14.5,26.5,28.0,14.5,4.0,4.0,13.0,5.5,16.0,17.5,20.5,25.0,15.0,4.5,11.0,6.0,19.0,6.0,5.5,13.5,24.5,24.0,19.5,15.0,5.0,18.0,16.0,17.5,14.5,28.5,18.0,13.5,20.0,4.5,28.0,22.5,9.5,12.5,20.5,25.0,25.0,14.5,3.0,4.0,12.5,14.0,16.5,26.0,18.5,18.5,19.5
55042,17.5,11.5,19.0,13.5,15.5,8.5,16.0,15.0,17.0,8.0,5.5,20.5,14.0,11.5,9.0,9.5,19.0,6.0,6.0,15.5,13.0,20.5,13.0,12.5,23.0,17.0,18.0,10.5,8.5,8.5,13.0,24.5,18.5,22.0,27.0,17.5,16.5,10.0,8.5,15.0,14.5,8.5,7.0,10.0,16.0,13.5,7.0,14.0,10.0,16.5,20.0,9.0,17.5,15.0,15.0,12.5
55043,19.0,17.5,15.5,18.0,17.5,15.5,24.0,25.0,7.5,4.5,7.0,11.5,3.5,12.5,18.5,16.0,18.5,9.5,7.0,11.5,9.5,14.5,8.0,8.0,14.5,27.0,26.0,21.0,17.5,10.0,11.5,18.0,26.5,18.0,23.0,17.0,13.5,17.5,8.5,26.0,21.5,10.0,16.0,15.0,21.0,23.5,13.0,4.0,6.0,9.0,17.5,18.0,27.0,17.5,17.5,16.0
55044,25.5,12.0,22.0,11.0,10.0,20.0,14.0,14.5,17.0,15.5,13.5,17.0,5.0,11.5,11.0,9.0,22.5,10.0,7.5,19.0,16.0,24.0,16.5,16.0,25.5,22.0,20.0,10.5,8.5,13.5,21.5,14.5,25.5,21.5,17.0,18.0,9.5,9.0,18.5,14.5,20.0,12.0,13.5,10.5,18.0,18.0,15.5,8.5,18.0,19.0,16.5,10.5,19.5,24.0,24.0,10.0


Make prior tables

In [33]:
run = 1
if run:
    DF_GAME_META = pd.read_csv('./data/2000-2022/facts_boxscores_meta.csv').drop(columns='Unnamed: 0')
    GAME_META = list(DF_GAME_META.columns)
    df_all_seasons_index = pd.read_csv('./data/2000-2022/facts_game_index.csv').drop(columns='Unnamed: 0')
    game_index      = pd.Index(map(tuple,df_all_seasons_index[['game_id','team_id']].values))
    team_prev_game_index    = pd.Index(map(tuple,df_all_seasons_index[['team_prev_game_id','team_id']].values))
    opp_prev_game_index     = pd.Index(map(tuple,df_all_seasons_index[['opp_prev_game_id','opp_id']].values))
    h2h_prev_game_index     = pd.Index(map(tuple,df_all_seasons_index[['h2h_prev_game_id','team_id']].values))
    opp_h2h_prev_game_index = pd.Index(map(tuple,df_all_seasons_index[['h2h_prev_game_id','opp_id']].values))

    df_records = pd.read_csv('./data/2000-2022/facts_cumu_win_records.csv').drop(columns='Unnamed: 0')
    df = df_records.set_index(game_index)
    columns = df.columns.drop(['season_h2h_win_pct'])

    for team_id in pd.unique(DF_GAME_META['team_id']):
        df.loc[(-1,team_id),:] = np.nan    # Create dummy index/entries

    team_prior      = df.loc[team_prev_game_index,columns].reset_index(drop=True)
    team_h2h_prior  = df.loc[h2h_prev_game_index,'season_h2h_win_pct'].to_frame().reset_index(drop=True)
    team_prior      = pd.concat([team_prior,team_h2h_prior],axis=1).fillna(0)
    opp_prior       = df.loc[opp_prev_game_index,columns].reset_index(drop=True)
    opp_h2h_prior   = df.loc[opp_h2h_prev_game_index,'season_h2h_win_pct'].to_frame().reset_index(drop=True)
    opp_prior       = pd.concat([opp_prior,opp_h2h_prior],axis=1).fillna(0)
    net_prior = (team_prior - opp_prior).fillna(0)
    avg_prior = ((team_prior + opp_prior)/2).fillna(0)

    team_prior.columns = team_prior.columns + '_team'
    opp_prior.columns = opp_prior.columns +  '_opp'
    net_prior.columns = net_prior.columns + '_net'
    avg_prior.columns = avg_prior.columns +  '_avg'
    team_prior.to_csv(f'./data/2000-2022/prior_cumu_win_records_team.csv')
    opp_prior.to_csv(f'./data/2000-2022/prior_cumu_win_records_opp.csv')
    net_prior.to_csv(f'./data/2000-2022/prior_cumu_win_records_net.csv')
    avg_prior.to_csv(f'./data/2000-2022/prior_cumu_win_records_avg.csv')

else:
    team_prior = pd.read_csv(f'./data/2000-2022/prior_cumu_win_records_team.csv').drop(columns='Unnamed: 0')
    opp_prior = pd.read_csv(f'./data/2000-2022/prior_cumu_win_records_opp.csv').drop(columns='Unnamed: 0')
    net_prior = pd.read_csv(f'./data/2000-2022/prior_cumu_win_records_net.csv').drop(columns='Unnamed: 0')
    avg_prior = pd.read_csv(f'./data/2000-2022/prior_cumu_win_records_avg.csv').drop(columns='Unnamed: 0')

print(f"[{dt.datetime.now().strftime('%H:%M:%S %d-%m-%Y')}] cached = {not bool(run)}")
display_html(team_prior)
display_html(opp_prior)
display_html(net_prior)
display_html(avg_prior)

[13:56:07 19-04-2023] cached = False


,home_win_pct_team,road_win_pct_team,loc_context_win_pct_team,same_conf_win_pct_team,cross_conf_win_pct_team,conf_context_win_pct_team,season_h2h_win_pct_team
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
2,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
3,1.000000,0.000000,1.000000,0.500000,1.000000,0.500000,0.000000
4,1.000000,0.500000,0.500000,0.500000,1.000000,1.000000,0.000000
...,...,...,...,...,...,...,...
55041,0.435897,0.289474,0.435897,0.367347,0.357143,0.367347,0.333333
55042,0.435897,0.282051,0.282051,0.360000,0.357143,0.360000,0.333333
55043,0.435897,0.300000,0.300000,0.372549,0.357143,0.372549,0.500000
55044,0.425000,0.300000,0.425000,0.365385,0.357143,0.365385,0.333333


,home_win_pct_opp,road_win_pct_opp,loc_context_win_pct_opp,same_conf_win_pct_opp,cross_conf_win_pct_opp,conf_context_win_pct_opp,season_h2h_win_pct_opp
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.333333,0.333333,0.500000,0.000000,0.500000,0.000000
4,1.000000,0.666667,0.666667,0.750000,0.000000,0.750000,0.000000
...,...,...,...,...,...,...,...
55041,0.710526,0.461538,0.710526,0.653061,0.464286,0.653061,0.666667
55042,0.307692,0.131579,0.307692,0.265306,0.142857,0.265306,0.666667
55043,0.550000,0.435897,0.550000,0.490196,0.500000,0.490196,0.500000
55044,0.625000,0.200000,0.625000,0.423077,0.392857,0.423077,0.666667


,home_win_pct_net,road_win_pct_net,loc_context_win_pct_net,same_conf_win_pct_net,cross_conf_win_pct_net,conf_context_win_pct_net,season_h2h_win_pct_net
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
2,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
3,1.000000,-0.333333,0.666667,0.000000,1.000000,0.000000,0.000000
4,0.000000,-0.166667,-0.166667,-0.250000,1.000000,0.250000,0.000000
...,...,...,...,...,...,...,...
55041,-0.274629,-0.172065,-0.274629,-0.285714,-0.107143,-0.285714,-0.333333
55042,0.128205,0.150472,-0.025641,0.094694,0.214286,0.094694,-0.333333
55043,-0.114103,-0.135897,-0.250000,-0.117647,-0.142857,-0.117647,0.000000
55044,-0.200000,0.100000,-0.200000,-0.057692,-0.035714,-0.057692,-0.333333


,home_win_pct_avg,road_win_pct_avg,loc_context_win_pct_avg,same_conf_win_pct_avg,cross_conf_win_pct_avg,conf_context_win_pct_avg,season_h2h_win_pct_avg
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.500000,0.000000,0.500000,0.000000,0.500000,0.500000,0.0
2,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,0.0
3,0.500000,0.166667,0.666667,0.500000,0.500000,0.500000,0.0
4,1.000000,0.583333,0.583333,0.625000,0.500000,0.875000,0.0
...,...,...,...,...,...,...,...
55041,0.573212,0.375506,0.573212,0.510204,0.410714,0.510204,0.5
55042,0.371795,0.206815,0.294872,0.312653,0.250000,0.312653,0.5
55043,0.492949,0.367949,0.425000,0.431373,0.428571,0.431373,0.5
55044,0.525000,0.250000,0.525000,0.394231,0.375000,0.394231,0.5
